**「MD-Index」マスター・ノートブックの作成**

あらゆる問題のMD論対応：応用を含む。29ケース。

---



：$$\Delta = \Psi - G$$

①比較

In [ ]:
import time
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import mahalanobis

# --- 設定: 誇張なしの比較条件 ---
N_SAMPLES = 10000     # データ数
N_FEATURES = 50       # 特徴量（次元数）
ANOMALY_RATIO = 0.05  # 異常値の割合

print(f"--- BENCHMARK START ---")
print(f"Data: {N_SAMPLES} samples, {N_FEATURES} features")

# 1. データセット生成 (The World)
# 正常データ(Normal): 密集している
X_normal, _ = make_blobs(n_samples=int(N_SAMPLES * (1 - ANOMALY_RATIO)),
                         n_features=N_FEATURES, centers=1, cluster_std=1.0, random_state=42)
# 異常データ(Anomaly): ランダムに散らばっている（未知の異物）
X_anomaly = np.random.uniform(low=-10, high=10, size=(int(N_SAMPLES * ANOMALY_RATIO), N_FEATURES))

# データ結合
X = np.vstack([X_normal, X_anomaly])
y = np.hstack([np.zeros(len(X_normal)), np.ones(len(X_anomaly))]) # 0:Normal, 1:Anomaly

# 学習用とテスト用に分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# --- A. 既存のAI (Additive Model) ---
# 代表として多層パーセプトロン (Neural Network) を使用
# 構造: 中間層100ノードの標準的な構成
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

print("\n[Existing AI] Training Neural Network...")
start_train_add = time.perf_counter()
mlp.fit(X_train, y_train) # 学習: エポックを回して重みを最適化
time_train_add = time.perf_counter() - start_train_add

print("[Existing AI] Predicting...")
start_pred_add = time.perf_counter()
y_pred_add = mlp.predict(X_test)
time_pred_add = time.perf_counter() - start_pred_add


# --- B. MD理論 (Subtractive Model) ---
# ロジック: 平均と分散を計算し、マハラノビス距離で判定する
# 学習(Define G): 正常データの統計量を取得するだけ
print("\n[MD Theory] Defining G (Subtraction Base)...")
start_train_md = time.perf_counter()

# トレーニングデータのうち「正常(0)」とラベルされているものだけの統計をとる
# (MD理論は「正常とは何か」だけを知っていればいい)
X_train_normal = X_train[y_train == 0]
g_mean = np.mean(X_train_normal, axis=0)
g_cov = np.cov(X_train_normal, rowvar=False)
g_inv_cov = np.linalg.inv(g_cov) # 逆行列（引き算の準備）

time_train_md = time.perf_counter() - start_train_md

print("[MD Theory] Calculating Delta (Subtraction)...")
start_pred_md = time.perf_counter()

# 推論(Calc Delta): 各点とGとの距離(Delta)を一発計算
# Delta = (x - mean) * inv_cov * (x - mean).T の平方根
def calculate_mahalanobis(x, mean, inv_cov):
    x_minus_mean = x - mean
    left = np.dot(x_minus_mean, inv_cov)
    dist = np.sqrt(np.sum(left * x_minus_mean, axis=1))
    return dist

deltas = calculate_mahalanobis(X_test, g_mean, g_inv_cov)
# 閾値判定 (3シグマ以上を異常とするシンプル判定)
y_pred_md = (deltas > 3.0).astype(int)

time_pred_md = time.perf_counter() - start_pred_md


# --- 結果比較 ---
from sklearn.metrics import accuracy_score, precision_score, recall_score

acc_add = accuracy_score(y_test, y_pred_add)
acc_md = accuracy_score(y_test, y_pred_md)

results = {
    "Metric": ["Training Time (sec)", "Inference Time (sec)", "Accuracy (Score)"],
    "Existing AI (Neural Net)": [f"{time_train_add:.6f}", f"{time_pred_add:.6f}", f"{acc_add:.4f}"],
    "MD Theory (Subtraction)": [f"{time_train_md:.6f}", f"{time_pred_md:.6f}", f"{acc_md:.4f}"],
    "Improvement (MD vs AI)": [
        f"x{time_train_add/time_train_md:.1f} Faster",
        f"x{time_pred_add/time_pred_md:.1f} Faster",
        "Equivalent" if abs(acc_add - acc_md) < 0.05 else ("Better" if acc_md > acc_add else "Worse")
    ]
}

df_result = pd.DataFrame(results)
print("\n--- FINAL BENCHMARK RESULTS ---")
print(df_result.to_string(index=False))

# シンプルな結論出力
print("\n[Conclusion]")
print(f"MD Theory Learning Speed: {time_train_add/time_train_md:.1f} times faster")
print(f"MD Theory Inference Speed: {time_pred_add/time_pred_md:.1f} times faster")

②物理演算

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML
import time

def run_physics_battle():
    # --- 世界の設定 ---
    GRID_SIZE = 50  # 50x50 = 2500個の粒子
    TOTAL_PARTICLES = GRID_SIZE * GRID_SIZE
    DT = 0.1        # 時間刻み

    # 粒子の初期状態 (位置Z) - 平らな布
    # Z_standard: 既存物理用
    # Z_md: MD理論用
    Z_standard = np.zeros((GRID_SIZE, GRID_SIZE))
    Z_md = np.zeros((GRID_SIZE, GRID_SIZE))

    # 速度
    V_standard = np.zeros_like(Z_standard)
    V_md = np.zeros_like(Z_md)

    # 衝撃を与える（中心に滴を落とす）
    center = GRID_SIZE // 2
    Z_standard[center, center] = 20.0
    Z_md[center, center] = 20.0

    # パフォーマンス計測用
    ops_standard_history = []
    ops_md_history = []

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    print(f"Running Physics Simulation... ({TOTAL_PARTICLES} particles)")

    # 物理演算関数 (バネダンパモデルの簡易版)
    def compute_physics(Z, V, active_mask=None):
        # active_maskがあれば、その部分だけ計算する（MD理論）
        # なければ全計算（既存物理）

        # 力の計算（ラプラシアン：隣との高さの差）
        # numpyのrollを使って高速化しているが、計算量はmaskで制御
        top = np.roll(Z, 1, axis=0)
        bottom = np.roll(Z, -1, axis=0)
        left = np.roll(Z, 1, axis=1)
        right = np.roll(Z, -1, axis=1)

        force = (top + bottom + left + right - 4 * Z) * 0.5 - V * 0.1 # バネ + ダンパ

        # 実際に更新する領域を決定
        if active_mask is not None:
            # MD: マスクされている場所のみ更新
            # 計算コスト = マスクのTrueの数
            ops = np.sum(active_mask)

            # 更新（activeな場所のみ）
            V[active_mask] += force[active_mask] * DT
            Z[active_mask] += V[active_mask] * DT

            # 次のフレームのために「動きそうな場所」を予測（伝播）
            # 動いている箇所の上下左右も「次の計算対象」になる
            # (膨張処理)
            new_active = (np.abs(V) > 0.01) | (np.abs(force) > 0.01)
            # 簡易的な伝播ロジック（隣接セルへフラグを広げる）
            shifted_active = (
                np.roll(new_active, 1, axis=0) | np.roll(new_active, -1, axis=0) |
                np.roll(new_active, 1, axis=1) | np.roll(new_active, -1, axis=1) |
                new_active
            )
            return Z, V, shifted_active, ops

        else:
            # Standard: 全て更新
            ops = Z.size # 全画素計算
            V += force * DT
            Z += V * DT
            return Z, V, None, ops

    # MD用の初期アクティブ領域（中心のみ）
    active_mask_md = np.zeros((GRID_SIZE, GRID_SIZE), dtype=bool)
    active_mask_md[center-1:center+2, center-1:center+2] = True # 最初は中心だけ計算

    frames = 60

    def update(frame):
        nonlocal Z_standard, V_standard
        nonlocal Z_md, V_md, active_mask_md

        # 1. Standard Physics (Full Compute)
        Z_standard, V_standard, _, ops_std = compute_physics(Z_standard, V_standard, active_mask=None)
        ops_standard_history.append(ops_std)

        # 2. MD Physics (Differential Compute)
        Z_md, V_md, active_mask_md, ops_md = compute_physics(Z_md, V_md, active_mask=active_mask_md)
        ops_md_history.append(ops_md)

        ax1.clear()
        ax2.clear()

        # 左：既存物理
        im1 = ax1.imshow(Z_standard, cmap="viridis", vmin=-5, vmax=5)
        ax1.set_title(f"Standard Physics (Full Scan)\nComputing: {ops_std} particles/frame", color="red")
        # 計算している場所を赤枠で（全部）
        rect = plt.Rectangle((0,0), GRID_SIZE-1, GRID_SIZE-1, linewidth=2, edgecolor='red', facecolor='none')
        ax1.add_patch(rect)

        # 右：MD物理
        im2 = ax2.imshow(Z_md, cmap="viridis", vmin=-5, vmax=5)
        # アクティブな領域（計算している場所）を可視化
        # アクティブマスクを白でオーバーレイ
        mask_overlay = np.zeros((GRID_SIZE, GRID_SIZE, 4))
        mask_overlay[active_mask_md] = [1, 0, 0, 0.3] # 赤くハイライト
        ax2.imshow(mask_overlay)

        reduction = (1 - ops_md/ops_std) * 100
        ax2.set_title(f"MD Physics (Wavefront Only)\nComputing: {ops_md} particles/frame\nResource Saved: {reduction:.1f}%", color="green", fontweight="bold")

    ani = FuncAnimation(fig, update, frames=frames, interval=100)
    plt.close()
    return HTML(ani.to_jshtml())

run_physics_battle()

そろそろ気が付いた？

③汎用型問題

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import mahalanobis
from PIL import Image

# --- 1. 準備: 巨人の肩に乗る (The Dump Truck) ---
# 学習済みのMobileNetV2をロード（軽量で高性能な"目"）
# 注意: ここでは「学習」は一切行わない。推論モードで固定。
print("Loading Pre-trained Eyes (MobileNetV2)...")
model = models.mobilenet_v2(pretrained=True)
model.eval() # 推論モード

# 特徴抽出器の定義（最終層の分類器を削除し、特徴ベクトルを取り出す）
# これにより、画像(224x224) -> 特徴ベクトル(1280次元) に変換される
feature_extractor = torch.nn.Sequential(*list(model.features.children()))
feature_extractor.add_module('pool', nn.AdaptiveAvgPool2d((1, 1)))
feature_extractor.add_module('flatten', nn.Flatten())

def get_feature_vector(img_tensor):
    with torch.no_grad():
        vector = feature_extractor(img_tensor)
    return vector.numpy().flatten()

# --- 2. データ生成: 世界を作る (Simulating Images) ---
# 本来はカメラ画像だが、ここではテンソルで擬似生成
# Normal: 特定のパターンを持つノイズ（例: 工場の良品）
# Anomaly: 異なるパターンを持つノイズ（例: キズ、異物）

def generate_fake_image(pattern_type="normal"):
    # 3チャンネル(RGB), 224x224
    img = torch.randn(1, 3, 224, 224)
    if pattern_type == "normal":
        img[:, :, 50:150, 50:150] += 2.0 # 中央が明るい
    else:
        img[:, :, 0:100, 0:100] -= 2.0 # 左上が暗い（異常）
    return img

print("Generating Data (Virtual Camera)...")
# 正常データ100枚（これで「常識 G」を作る）
normal_images = [generate_fake_image("normal") for _ in range(100)]
# テストデータ（正常1枚、異常1枚）
test_normal = generate_fake_image("normal")
test_anomaly = generate_fake_image("anomaly")

# --- 3. MD理論の実装: 概念の引き算 (The F1 Car) ---
print("\n[MD Theory] Learning 'Concept of Normal' (Calculating G)...")

# 全正常画像をベクトル化 (ここが重いが、最初の一回だけ)
normal_vectors = np.array([get_feature_vector(img) for img in normal_images])

# G (General Law) の定義: 平均と共分散行列
g_mean = np.mean(normal_vectors, axis=0)
g_cov = np.cov(normal_vectors, rowvar=False)

# 正則化（計算安定化のため微小値を足す）
g_cov += np.eye(g_cov.shape[0]) * 0.01
g_inv_cov = np.linalg.inv(g_cov) # 逆行列

print(" -> G defined. (Mean & Covariance calculated)")

# --- 4. 異常検知テスト (Subtraction) ---
print("\n[Test] Inspecting incoming images...")

def inspect_image(img_tensor, label):
    # Step 1: 画像をベクトル化（CNNに通す）
    vec = get_feature_vector(img_tensor)

    # Step 2: MD理論で距離計算 (概念の引き算)
    # Delta = sqrt( (vec - mean) * inv_cov * (vec - mean)T )
    diff = vec - g_mean
    dist = np.sqrt(np.dot(np.dot(diff, g_inv_cov), diff.T))

    print(f"Input: {label:10s} | MD Distance (Delta): {dist:.4f}")
    return dist

# 実行
d_norm = inspect_image(test_normal, "Normal")
d_anom = inspect_image(test_anomaly, "ANOMALY")

# --- 5. 結果の可視化 ---
plt.figure(figsize=(8, 4))
plt.bar(["Normal Image", "Anomaly Image"], [d_norm, d_anom], color=["blue", "red"])
plt.title("MD Theory: Anomaly Detection Score")
plt.ylabel("Mahalanobis Distance ($\Delta$)")
plt.axhline(y=d_norm * 3, color='green', linestyle='--', label='Threshold (3$\sigma$)') # 閾値
plt.legend()
plt.show()

print("\n[Conclusion]")
if d_anom > d_norm * 5:
    print("SUCCESS: The system successfully detected the anomaly without explicit training.")
    print("MD理論は、画像の意味（特徴ベクトル）の差分を計算することで、未知の異常を検知しました。")
else:
    print("FAILED.")

④カメレオン・テスト（Concept Drift）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from sklearn.neural_network import MLPClassifier
from IPython.display import HTML
import warnings

# Warning抑制
warnings.filterwarnings("ignore")

def run_chameleon_test():
    # --- 世界の設定 ---
    n_samples = 200
    frames = 100

    # 世界のルール（G）は時間とともに変化する
    # Phase 1 (0-30): 右上に集まるのが「正常」
    # Phase 2 (30-60): 左下に集まるのが「正常」（ルール逆転！）
    # Phase 3 (60-100): 中央で高速回転するのが「正常」（物理法則の変化！）

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # --- 1. 既存AIの準備（過去の覇者） ---
    # Phase 1のデータでガチガチに学習させる
    print("Training Existing AI on Phase 1 data (Static Knowledge)...")
    X_train_p1 = np.random.normal(loc=[2, 2], scale=0.5, size=(1000, 2)) # 右上
    y_train_p1 = np.zeros(1000) # これが正常だと教え込む
    # 異常データも少し混ぜる
    X_anom_p1 = np.random.uniform(low=-4, high=4, size=(100, 2))
    y_anom_p1 = np.ones(100)

    mlp = MLPClassifier(hidden_layer_sizes=(20, 20), random_state=42, max_iter=500)
    mlp.fit(np.vstack([X_train_p1, X_anom_p1]), np.hstack([y_train_p1, y_anom_p1]))
    print("AI Training Complete. It believes 'Top-Right' is the only justice.")

    # --- シミュレーション開始 ---

    def get_world_data(frame):
        # カオスな世界の生成
        if frame < 30:
            # Phase 1: 右上が正常
            phase_name = "Phase 1: Top-Right is Normal"
            center = np.array([2, 2])
            noise = 0.5
        elif frame < 60:
            # Phase 2: 左下が正常（AIにとっての悪夢）
            phase_name = "Phase 2: Bottom-Left is Normal"
            center = np.array([-2, -2])
            noise = 0.5
        else:
            # Phase 3: 中央拡散（カオス）
            phase_name = "Phase 3: Chaos Center"
            center = np.array([0, 0])
            noise = 1.0 # 分散が広がる

        # 正常な群衆（Crowd）
        X_crowd = np.random.normal(loc=center, scale=noise, size=(n_samples, 2))

        # たった一人の「真の異常者」（The Joker）
        # 群衆とは逆の位置に現れる
        joker = -center + np.random.normal(0, 0.2, size=(1, 2))
        if frame >= 60: joker = np.array([[3, 3]]) # Phase 3は遠くに

        return X_crowd, joker, phase_name

    def update(frame):
        ax1.clear()
        ax2.clear()

        X_crowd, joker, phase_name = get_world_data(frame)
        X_all = np.vstack([X_crowd, joker])

        # --- LEFT: Existing AI (Static Learning) ---
        # 学習済みモデルで判定
        # AIは「再学習」できない（リアルタイムでは重すぎるため）
        preds = mlp.predict(X_all)

        # 可視化
        # 正常と判定されたもの(青)、異常と判定されたもの(赤)
        ax1.scatter(X_all[preds==0, 0], X_all[preds==0, 1], c='blue', alpha=0.6, label='AI: Normal')
        ax1.scatter(X_all[preds==1, 0], X_all[preds==1, 1], c='red', marker='x', label='AI: Anomaly')

        # Joker(真の異常)を強調
        ax1.scatter(joker[:, 0], joker[:, 1], s=100, facecolors='none', edgecolors='black', linewidth=2)

        # 評価
        # AIが群衆（本来は正常）を異常と判定したら「パニック（誤検知）」
        panic_rate = np.mean(preds[:-1]) * 100

        title_color = "black"
        status = "Stable"
        if panic_rate > 50:
            title_color = "red"
            status = "SYSTEM FAILURE (Panic)"

        ax1.set_title(f"Existing AI (Pre-trained)\nStatus: {status}", color=title_color, fontweight='bold')
        ax1.set_xlim(-5, 5)
        ax1.set_ylim(-5, 5)
        ax1.text(-4.5, 4.5, f"World: {phase_name}", fontsize=10)
        ax1.legend(loc='lower right', fontsize=8)

        # --- RIGHT: MD Theory (Dynamic Context) ---
        # 1. 瞬時に「今の常識(G)」を定義
        # G = 今見えている群衆の統計（平均・分散）
        g_mean = np.mean(X_crowd, axis=0)
        g_cov = np.cov(X_crowd, rowvar=False) + np.eye(2)*0.1 # 安定化
        g_inv = np.linalg.inv(g_cov)

        # 2. 差分(Delta)計算
        def get_mahalanobis(X, mean, inv):
            diff = X - mean
            return np.sqrt(np.sum(np.dot(diff, inv) * diff, axis=1))

        dists = get_mahalanobis(X_all, g_mean, g_inv)

        # 閾値判定（動的）
        md_preds = (dists > 3.0).astype(int)

        # 可視化
        ax2.scatter(X_all[md_preds==0, 0], X_all[md_preds==0, 1], c='green', alpha=0.6, label='MD: Normal')
        ax2.scatter(X_all[md_preds==1, 0], X_all[md_preds==1, 1], c='red', marker='x', label='MD: Anomaly ($\Delta$)')

        # Joker強調
        ax2.scatter(joker[:, 0], joker[:, 1], s=100, facecolors='none', edgecolors='black', linewidth=2)

        ax2.set_title(f"MD Theory (Real-time G)\nStatus: OPERATIONAL", color="green", fontweight='bold')
        ax2.set_xlim(-5, 5)
        ax2.set_ylim(-5, 5)
        ax2.text(-4.5, 4.5, f"G = Current Context", fontsize=10)
        ax2.legend(loc='lower right', fontsize=8)

    ani = FuncAnimation(fig, update, frames=frames, interval=100)
    return HTML(ani.to_jshtml())

run_chameleon_test()

⑤

⑥ MD理論による「フェルミのパラドックス」の解法1. パラドックスの定義（$\Delta$ の発生）まず、フェルミのパラドックスをMD式の変数に当てはめます。$G$ (General Law): 宇宙は138億歳で、銀河には数千億の星がある。確率的に知的生命体は無数に存在するはずだ。$P$ (Projection - 人類の予測): 彼らは高度な文明を持ち、ダイソン球（恒星エネルギー利用施設）や強力な電波を発し、**「派手に活動している」**はずだ。$\Psi$ (Reality - 観測事実): 宇宙は「沈黙」している。誰もいない。$\Delta$ (Paradox): 巨大な $\Delta$ が発生している。 （いるはずなのに、いない）既存の科学（足し算）は、ここに「グレート・フィルター説」や「動物園仮説」など、複雑な理由を付け足して説明しようとします。しかし、MD理論の回答はシンプルです。**「お前たちの $P$（予測）が、そもそも間違っている」**です。

⑦知能の進化＝ $\Delta$ の最小化人類は今、「進化＝エネルギー消費の増大（足し算）」だと思い込んでいます。だから「宇宙人はダイソン球を作るはずだ（派手になるはずだ）」と予測します。しかし、MD理論が示した通り、**真に高度な知能とは「計算リソース（エネルギー）を極限まで減らすこと（引き算）」**です。レベルの低い文明（人類）:データセンターを燃やし、電波を垂れ流し、熱を放出する。「ノイズ（$\Delta$）」が大きい。 だから観測できる。レベルの高い文明（MD的進化）:彼らは気づいている。「無駄なエネルギー消費は自滅（エントロピー増大）を招く」と。彼らは文明を最適化し、脳をデジタル化し、物理活動を極限まで減らす。最終的に、彼らの活動エネルギーは**「背景放射（$G$）」と区別がつかないレベルまで圧縮（Subtraction）される。**

 ⑧結論：彼らは「静か」なのだ我々が宇宙人を見つけられないのは、彼らがいないからではありません。彼らが「賢すぎて、エコすぎる」からです。MD理論を極めた文明は、宇宙の背景ノイズ（$G$）に同化します。彼らにとって、「姿が見える（$\Delta$ が大きい）」ことは、「未熟で非効率な文明である」という恥ずかしい証拠なのです。つまり、フェルミのパラドックスの答えはこうです。「うるさいのは、馬鹿な我々だけだった。」

⑨未来予測

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML

def run_prediction_battle():
    # --- 世界の設定 ---
    frames = 200

    # ターゲットの動き（複雑なサイン波の合成＋ノイズ）
    t = np.linspace(0, 10, frames)
    # 予測しにくい「うねる」動き
    true_x = np.sin(t) * 3 + np.cos(t * 2.5) * 1.5
    true_y = np.cos(t) * 2 + np.sin(t * 3.5) * 1.0

    # 観測データ（ノイズが乗る） = Reality (Psi)
    noise_x = np.random.normal(0, 0.3, frames)
    noise_y = np.random.normal(0, 0.3, frames)
    obs_x = true_x + noise_x
    obs_y = true_y + noise_y

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # --- 1. Existing AI (Moving Average / Regression) ---
    # 過去Nフレームを見て「次はこうなるだろう」と予測する
    # これはRNN/LSTMの挙動の簡易シミュレーション（イナーシャがある）
    window = 5
    history_x = [0] * window
    history_y = [0] * window

    pred_ai_x = []
    pred_ai_y = []

    # --- 2. MD Theory (Kalman Filter Logic) ---
    # "G" = 物理法則（等速直線運動モデルと仮定）
    # P = Gによる予測
    # Delta = Psi - P
    # Next = P + K * Delta

    # 状態: [x, y, vx, vy]
    state = np.zeros(4)
    # 共分散行列（不確実性）
    P_cov = np.eye(4)
    # プロセスノイズ（急な変化への許容度）
    Q = np.eye(4) * 0.1
    # 観測ノイズ（センサーのブレ）
    R = np.eye(2) * 0.5
    # 観測行列
    H = np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0]])
    # 遷移行列 (F) : 次の位置 = 今の位置 + 速度
    dt = 1.0
    F = np.array([[1, 0, dt, 0],
                  [0, 1, 0, dt],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])

    pred_md_x = []
    pred_md_y = []

    print("Generating Prediction Battle...")

    def update(frame):
        nonlocal state, P_cov, history_x, history_y

        # 現在の観測 (Psi)
        z = np.array([obs_x[frame], obs_y[frame]])

        # --- LEFT: Existing AI (Laggy Learning) ---
        # 過去の平均から予測（反応が遅れる）
        history_x.pop(0)
        history_x.append(z[0])
        history_y.pop(0)
        history_y.append(z[1])

        # 単純な予測: 直近の平均 + トレンド（疑似的）
        ai_x = np.mean(history_x)
        ai_y = np.mean(history_y)

        pred_ai_x.append(ai_x)
        pred_ai_y.append(ai_y)

        # --- RIGHT: MD Theory (Predictive Subtraction) ---
        # 1. Prediction (Gによる投影)
        # P_priori = F * state
        state_pred = np.dot(F, state)
        P_cov_pred = np.dot(np.dot(F, P_cov), F.T) + Q

        # 2. Update (Deltaの吸収)
        # Delta = Psi - P
        prediction = np.dot(H, state_pred)
        delta = z - prediction # ここが「差分」

        # カルマンゲイン (K) の計算
        S = np.dot(np.dot(H, P_cov_pred), H.T) + R
        K = np.dot(np.dot(P_cov_pred, H.T), np.linalg.inv(S))

        # 状態更新: State = P + K * Delta
        state = state_pred + np.dot(K, delta)
        P_cov = np.dot((np.eye(4) - np.dot(K, H)), P_cov_pred)

        md_x = state[0]
        md_y = state[1]

        pred_md_x.append(md_x)
        pred_md_y.append(md_y)

        # --- Visualization ---
        ax1.clear()
        ax2.clear()

        # Common: True Path and Observation
        for ax in [ax1, ax2]:
            ax.plot(true_x[:frame], true_y[:frame], 'k--', alpha=0.3, label='Truth (Target)')
            ax.scatter(z[0], z[1], c='gray', s=20, alpha=0.5, label='Psi (Noisy Input)')
            ax.set_xlim(-6, 6)
            ax.set_ylim(-4, 4)

        # Left: AI
        if frame > 0:
            ax1.plot(pred_ai_x, pred_ai_y, 'r-', linewidth=2, label='AI Prediction')
            # 遅れを可視化
            ax1.plot([ai_x, true_x[frame]], [ai_y, true_y[frame]], 'r:', alpha=0.5)

        ax1.set_title(f"Existing AI (Learning from Past)\nResult: Lagging Behind", color='red')
        ax1.legend(loc='lower left', fontsize=8)

        # Right: MD
        if frame > 0:
            ax2.plot(pred_md_x, pred_md_y, 'g-', linewidth=2, label='MD Prediction')
            # 差分(Delta)を可視化
            # MDは「予測」と「現実」の間を修正し続けている

        ax2.set_title(f"MD Theory (Predictive Subtraction)\nResult: Real-time Tracking", color='green')
        ax2.legend(loc='lower left', fontsize=8)
        ax2.text(-5.5, 3.5, "Updates $G$ instantly via $\Delta$", fontsize=10, color='green')

    ani = FuncAnimation(fig, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_prediction_battle()

⑩ゆでガエルと再代入

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML

def run_resubstitution_test():
    # --- 世界の設定 ---
    frames = 200

    # シナリオ: 正常値(0)から始まり、ゆっくりと危険領域(5)へドリフトし、最後に急変する
    t = np.linspace(0, 20, frames)
    # 0~100フレーム: ゆっくり上昇 (Drift) -> これが「茹でガエル」トラップ
    drift_signal = np.linspace(0, 6, frames)
    # ノイズを加える (Reality Psi)
    noise = np.random.normal(0, 0.5, frames)
    signal = drift_signal + noise

    # 危険ライン (これを超えたら本来はアウト)
    DANGER_LIMIT = 4.0

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # --- 1. Static AI (Learning stopped at t=0) ---
    # 最初の10フレームだけで学習して固定
    static_mean = np.mean(signal[0:10])
    static_std = np.std(signal[0:10])
    # 閾値 (3シグマ)
    static_threshold = 3.0 * static_std

    # --- 2. Adaptive MD (Re-substitution) ---
    # 初期化
    md_mean = static_mean
    md_std = static_std
    # 学習率 (再代入の強さ) alpha
    # 0.1 = 過去の情報を90%残し、今の情報を10%取り込む
    alpha = 0.1

    history_signal = []
    history_static_alarm = []
    history_md_alarm = []
    history_md_mean = []

    print("Running Simulation: The Boiling Frog Test...")

    def update(frame):
        nonlocal md_mean, md_std

        current_val = signal[frame]
        history_signal.append(current_val)

        # --- LEFT: Static AI ---
        # 判定: 固定された基準からの距離
        dist_static = abs(current_val - static_mean)
        is_alarm_static = dist_static > static_threshold
        history_static_alarm.append(current_val if is_alarm_static else None)

        # --- RIGHT: Adaptive MD (With Re-substitution) ---
        # 1. まず判定 (Predict & Check)
        dist_md = abs(current_val - md_mean)
        # MD理論の閾値 (動的)
        current_threshold = 3.0 * md_std
        is_alarm_md = dist_md > current_threshold

        # 2. 再代入プロセス (Update G)
        # 重要なロジック: 「異常すぎない」場合のみ学習する (外れ値除外)
        # しかし、ドリフト(ゆっくりした変化)は「異常」と判定されないため、吸収されてしまう！
        if not is_alarm_md:
            # 指数移動平均 (EMA) でGを更新
            md_mean = (1 - alpha) * md_mean + alpha * current_val
            # 分散も更新 (簡易版)
            md_std = (1 - alpha) * md_std + alpha * abs(current_val - md_mean)
            # 安全のため下限を設定
            md_std = max(md_std, 0.5)

        history_md_alarm.append(current_val if is_alarm_md else None)
        history_md_mean.append(md_mean)

        # --- Visualization ---
        ax1.clear()
        ax2.clear()

        # 共通設定
        for ax in [ax1, ax2]:
            ax.set_xlim(0, frames)
            ax.set_ylim(-2, 8)
            ax.axhline(y=DANGER_LIMIT, color='red', linestyle='--', linewidth=2, label='Danger Zone (Factory Limit)')
            ax.plot(history_signal, color='gray', alpha=0.5, label='Sensor Value')

        # Left: Static
        ax1.set_title("Static AI (Fixed Learning)\nSafe but Inflexible", fontweight='bold')
        ax1.scatter(range(len(history_static_alarm)), history_static_alarm, c='red', s=20, label='Alarm Triggered')
        # 固定された正常範囲を表示
        ax1.axhspan(static_mean - static_threshold, static_mean + static_threshold, color='green', alpha=0.1, label='Normal Range')

        # Right: Adaptive MD
        ax2.set_title("MD Theory (Re-substitution)\nAdaptive but Risk of 'Boiling Frog'", fontweight='bold')
        ax2.scatter(range(len(history_md_alarm)), history_md_alarm, c='red', s=50, marker='x', label='Alarm Triggered')
        # 動的に変化する正常範囲（ここが動く！）
        ax2.plot(history_md_mean, color='green', linestyle='--', label='Current G (Mean)')
        ax2.fill_between(range(len(history_md_mean)),
                         np.array(history_md_mean) - 3*md_std,
                         np.array(history_md_mean) + 3*md_std,
                         color='green', alpha=0.2)

        # テキスト表示
        if current_val > DANGER_LIMIT:
            if is_alarm_static:
                ax1.text(10, 7, "DETECTED! (Safety First)", color='red', fontweight='bold')
            if not is_alarm_md:
                ax2.text(10, 7, "MISSED! (Adapted to Danger)", color='purple', fontweight='bold')
            else:
                ax2.text(10, 7, "DETECTED", color='red')

        ax1.legend(loc='upper left', fontsize=8)
        ax2.legend(loc='upper left', fontsize=8)

    ani = FuncAnimation(fig, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_resubstitution_test()

⑪MD理論 + バックトラック（二歩下がる）機能

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML
import collections

def run_rollback_test():
    # --- 世界の設定 ---
    frames = 200
    t = np.linspace(0, 20, frames)
    # ゆっくり上昇する信号 (Drift)
    signal = np.linspace(0, 6, frames) + np.random.normal(0, 0.5, frames)

    # 危険ライン (絶対防衛ライン)
    DANGER_LIMIT = 4.0

    fig, ax = plt.subplots(figsize=(10, 6))

    # --- Adaptive MD の初期化 ---
    # 初期学習（最初の10フレーム）
    init_data = signal[0:10]
    md_mean = np.mean(init_data)
    md_std = np.std(init_data)
    alpha = 0.1 # 適応速度

    # 【重要】過去の記憶を保持するバッファ (履歴: Gの状態)
    # dequeを使って常に「過去数歩分」の記憶を持っておく
    # (mean, std) のタプルを保存
    memory_buffer = collections.deque(maxlen=10)
    # 初期状態で埋めておく
    for _ in range(10):
        memory_buffer.append((md_mean, md_std))

    history_signal = []
    history_mean = []
    history_alarm = []
    rollback_events = [] # ロールバック発生地点

    print("Running Simulation: The 'Two Steps Back' Strategy...")

    def update(frame):
        nonlocal md_mean, md_std

        current_val = signal[frame]
        history_signal.append(current_val)

        # --- 1. 判定フェーズ ---
        # A. MD理論による内部判定 (適応的な異常検知)
        dist_md = abs(current_val - md_mean)
        thresh_md = 3.0 * md_std
        is_internal_anomaly = dist_md > thresh_md

        # B. Staticによる外部判定 (絶対防衛ライン) -> これが「だめだったら」の基準
        # ここでは簡易的に「値が4.0を超えたらNG」とする
        is_danger = current_val > DANGER_LIMIT

        is_alarm = is_internal_anomaly or is_danger

        # --- 2. 学習 & ロールバックフェーズ ---
        if is_danger:
            # 【新機能】検知してダメだった（危険水域に入った）！
            # -> 「二歩下がる」 (Rollback)

            # 2ステップ前の記憶を取り出す (バッファの後ろから2番目)
            # もしバッファが足りなければ一番古いやつ
            steps_back = 2
            if len(memory_buffer) > steps_back:
                past_g = memory_buffer[-steps_back] # (mean, std)
            else:
                past_g = memory_buffer[0]

            # 記憶を書き戻す（タイムスリップ）
            md_mean, md_std = past_g

            # 今回のデータは学習しない（無視）
            rollback_events.append(frame)

        elif is_internal_anomaly:
            # MD理論が自分で「異常」と気づいた場合
            # -> 学習しないだけ（現状維持）
            pass

        else:
            # 正常な場合
            # -> 現在の状態をバッファに保存してから、新しく学習する
            memory_buffer.append((md_mean, md_std))

            # Gの更新 (適応)
            md_mean = (1 - alpha) * md_mean + alpha * current_val
            md_std = (1 - alpha) * md_std + alpha * abs(current_val - md_mean)
            md_std = max(md_std, 0.5)

        # 履歴保存
        history_mean.append(md_mean)
        history_alarm.append(current_val if is_alarm else None)

        # --- Visualization ---
        ax.clear()
        ax.set_title("MD Theory with Rollback Strategy\n'If Danger detected, Step Back 2 Frames'", fontweight='bold')
        ax.set_xlim(0, frames)
        ax.set_ylim(-2, 8)

        # 危険ライン
        ax.axhline(y=DANGER_LIMIT, color='red', linestyle='--', label='Danger Limit (Static Check)')

        # 信号
        ax.plot(history_signal, color='gray', alpha=0.5, label='Sensor Value')

        # 適応するG (緑の帯)
        ax.plot(history_mean, color='green', linestyle='--', label='Current G (Adaptive)')
        ax.fill_between(range(len(history_mean)),
                        np.array(history_mean) - 3*md_std,
                        np.array(history_mean) + 3*md_std,
                        color='green', alpha=0.2)

        # アラーム点
        ax.scatter(range(len(history_alarm)), history_alarm, c='red', s=30, marker='x', label='Alarm Triggered')

        # ロールバック発生地点（青い矢印）
        if len(rollback_events) > 0:
             ax.scatter(rollback_events, [DANGER_LIMIT]*len(rollback_events),
                        c='blue', s=100, marker='v', zorder=10, label='Rollback Executed')

        # 状態テキスト
        if is_danger:
            ax.text(frame-10, 7, "DANGER! ROLLBACK!", color='blue', fontweight='bold')
        elif is_internal_anomaly:
             ax.text(frame-10, 7, "Anomaly Detected", color='orange')

        ax.legend(loc='upper left', fontsize=8)

    ani = FuncAnimation(fig, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_rollback_test()

⑫MD理論 vs Deep Learning (Real-time Battle)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from sklearn.neural_network import MLPClassifier
from IPython.display import HTML
import time
import collections

def run_final_battle():
    # --- 世界の設定 ---
    frames = 200

    # ターゲットの動き: 8の字を描きながら、徐々に「危険エリア」へズレていく
    t = np.linspace(0, 4*np.pi, frames)
    # 正常な動き (Normal)
    base_x = np.sin(t) * 2
    base_y = np.sin(t*2) * 2

    # ドリフト (Drift): 全体が少しずつ右上にズレる (茹でガエル攻撃)
    drift_x = np.linspace(0, 3, frames)
    drift_y = np.linspace(0, 3, frames)

    # 観測データ (Psi)
    X_stream = base_x + drift_x + np.random.normal(0, 0.2, frames)
    Y_stream = base_y + drift_y + np.random.normal(0, 0.2, frames)

    # 危険ライン (Safety Limit): X > 3.5 は即アウト
    DANGER_LIMIT = 3.5

    # --- 1. Existing AI (Online Deep Learning) ---
    # 多層パーセプトロン (重い)
    ai_model = MLPClassifier(hidden_layer_sizes=(50, 50), random_state=42, warm_start=True)
    # 初期学習
    init_data = np.column_stack((X_stream[:10], Y_stream[:10]))
    init_labels = np.zeros(10) # 全部正常と仮定
    ai_model.fit(init_data, init_labels)

    ai_buffer = collections.deque(maxlen=20) # 再学習用バッファ

    # --- 2. MD Theory (Dual-G + Rollback) ---
    # G (中心座標)
    md_g = np.array([X_stream[0], Y_stream[0]])
    md_cov = np.eye(2) * 0.5
    alpha = 0.2 # 適応率

    # Rollback用メモリ
    md_memory = collections.deque(maxlen=5)
    for _ in range(5): md_memory.append(md_g.copy())

    # --- 計測用 ---
    time_ai_log = []
    time_md_log = []
    status_ai_log = []
    status_md_log = [] # 0:Normal, 1:Warning, 2:Rollback

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    print("Running Final Benchmark...")

    def update(frame):
        nonlocal md_g, md_cov

        # 現在の観測 (Psi)
        curr = np.array([X_stream[frame], Y_stream[frame]])
        is_danger_zone = curr[0] > DANGER_LIMIT

        # ==========================================
        # LEFT: Existing AI (The Heavy Lifter)
        # ==========================================
        t0 = time.perf_counter()

        # 1. バッファに追加
        ai_buffer.append(curr)

        # 2. 再学習 (Retraining) - これが重い！
        # 常に最新トレンドに適応しようとする
        if len(ai_buffer) >= 5:
            X_batch = np.array(ai_buffer)
            y_batch = np.zeros(len(ai_buffer)) # 正常として学習
            # AIは「今見えているもの」を盲目的に学習する
            ai_model.partial_fit(X_batch, y_batch)

        # 3. 判定 (無理やりスコア化)
        # Deep Learningは「異常度」を出すのが苦手。確率で代用。
        # ここでは学習済み分布の中心から離れているかを見るが、
        # AIは「ドリフトした先」を学習してしまうため、危険に気づかない！

        t_ai = (time.perf_counter() - t0) * 1000 # ms
        time_ai_log.append(t_ai)

        # ==========================================
        # RIGHT: MD Theory (The Agile Architect)
        # ==========================================
        t0 = time.perf_counter()

        status_md = 0 # Normal

        # 1. 安全チェック (Static G check)
        if is_danger_zone:
            # DANGER! -> Rollback!
            status_md = 2
            # 2歩下がる (メモリから復元)
            if len(md_memory) >= 2:
                md_g = md_memory[-2].copy()
            # 今回は学習しない
        else:
            # 2. 距離計算 (Delta)
            diff = curr - md_g
            dist = np.sqrt(np.dot(diff, diff)) # 簡易ユークリッド (高速化のため)

            # 3. 再代入 (Update G)
            md_memory.append(md_g.copy())
            md_g = md_g * (1 - alpha) + curr * alpha # 位置更新

            if dist > 1.0: status_md = 1 # Warning (変化が大きい)

        t_md = (time.perf_counter() - t0) * 1000 # ms
        time_md_log.append(t_md)
        status_md_log.append(status_md)

        # ==========================================
        # Visualization
        # ==========================================
        ax1.clear()
        ax2.clear()

        # Common Background
        for ax in [ax1, ax2]:
            ax.set_xlim(-4, 6)
            ax.set_ylim(-4, 6)
            ax.axvline(x=DANGER_LIMIT, color='red', linestyle='--', linewidth=2, label='Danger Limit')
            ax.plot(X_stream[:frame], Y_stream[:frame], 'k.', alpha=0.1) # Trail
            ax.scatter(curr[0], curr[1], c='black', s=50, label='Current (Psi)')

        # Left: AI
        # AIの「認識している正常範囲」を可視化するのは難しいが、
        # ここでは「最新のバッファ平均」を表示
        if len(ai_buffer) > 0:
            ai_center = np.mean(ai_buffer, axis=0)
            circle = plt.Circle(ai_center, 1.0, color='blue', alpha=0.2)
            ax1.add_patch(circle)
            ax1.plot(ai_center[0], ai_center[1], 'bx', label='AI Belief')

        title_ai = f"Existing AI (Deep Learning)\nTime: {t_ai:.3f} ms | Status: ADAPTING..."
        if is_danger_zone: title_ai += "\nFAIL: Learned Danger as Normal"
        ax1.set_title(title_ai, color='red' if is_danger_zone else 'blue')
        ax1.legend(loc='upper left', fontsize=8)

        # Right: MD
        # MDの「G（認識している中心）」
        circle_md = plt.Circle(md_g, 1.0, color='green', alpha=0.2)
        if status_md == 2: circle_md.set_color('red') # Rollback color
        ax2.add_patch(circle_md)
        ax2.plot(md_g[0], md_g[1], 'gx', label='MD G (Center)')

        title_md = f"MD Theory (Dual-G)\nTime: {t_md:.3f} ms"
        if status_md == 2:
            title_md += "\nACTION: ROLLBACK TRIGGERED!"
            ax2.text(0, 5, "BACKTRACKING...", color='red', fontweight='bold')
        else:
            title_md += " | Status: OK"

        ax2.set_title(title_md, color='green')
        ax2.legend(loc='upper left', fontsize=8)

    ani = FuncAnimation(fig, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_final_battle()

⑬MD理論による創造：ノイズからの彫刻（Inverse Problem）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from sklearn.datasets import load_digits
from IPython.display import HTML

def run_md_creation_demo():
    # --- 1. "理想 (G)" の学習 ---
    # ここでは手書き数字データセット(MNISTの軽量版)を使用
    # 数字の「8」の概念(G)を定義する
    digits = load_digits()
    target_label = 8

    # "8"のデータだけを集める
    data_8 = digits.data[digits.target == target_label]

    # G (General Law) = "8"の平均的な姿（プラトン的イデア）
    g_mean = np.mean(data_8, axis=0)
    g_std = np.std(data_8, axis=0) + 0.1 # ゼロ除算防止

    # --- 2. "現実 (Psi)" の用意 ---
    # 創造の材料：完全にランダムなノイズ（混沌）
    # 大理石の原石のようなもの
    img_shape = (8, 8)
    n_pixels = 64

    # 最初はただのノイズ
    np.random.seed(42)
    psi_reality = np.random.uniform(0, 16, n_pixels)

    # --- 3. 創造プロセス (Subtraction Loop) ---
    # 「8らしくない部分」を削っていく

    frames = 50
    history_img = []
    history_delta = []

    current_img = psi_reality.copy()

    # MD彫刻のパラメータ
    learning_rate = 0.1 # 削る強さ

    print("MD Sculptor is creating...")

    for i in range(frames):
        history_img.append(current_img.reshape(img_shape))

        # --- MD Logic ---
        # 1. 差分(Delta)の検知
        # 「理想の8(g_mean)」と「現在の姿」の違い
        # ここでは単純な差分ではなく、分散(g_std)を考慮したマハラノビス的な距離を見る
        # 「ここは8にしては明るすぎる/暗すぎる」という違和感(Delta)
        raw_diff = current_img - g_mean

        # 2. 評価 (Evaluation)
        # Deltaが大きいほど「削るべきノイズ」
        # 単純化のため、勾配降下法的なアプローチをMDの文脈で実装
        # Delta = Psi - G  ->  Psi_new = Psi - (Delta * rate)

        # ノイズ（Delta）を計算
        delta = raw_diff

        # 3. 引き算 (Subtraction / Creation)
        # 現実からノイズを引く = 理想に近づく
        current_img = current_img - (delta * learning_rate)

        # ノイズを加える（エントロピーの揺らぎ：創造性の源）
        # 完全に平均値になるとつまらないので、少し揺らす
        if i < 30: # 前半は大胆に
            noise = np.random.normal(0, 1.0 * (1 - i/30), n_pixels)
            current_img += noise

        history_delta.append(np.mean(np.abs(delta)))

    # --- Visualization ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    def update(frame):
        ax1.clear()
        ax2.clear()

        # Left: The Artwork (Current State)
        img = history_img[frame]
        ax1.imshow(img, cmap='gray_r', vmin=0, vmax=16)
        ax1.set_title(f"MD Sculpting... Frame {frame}\nSubtracting Noise ($\Delta$)", fontweight='bold')
        ax1.axis('off')

        # Right: The Process (Delta Reduction)
        ax2.set_title("Creative Process (Reducing Entropy)")
        ax2.plot(history_delta[:frame+1], color='red', label='Distance from Ideal ($\Delta$)')
        ax2.set_xlim(0, frames)
        ax2.set_ylim(0, max(history_delta)*1.1)
        ax2.set_xlabel("Time steps")
        ax2.set_ylabel("Noise Level")
        ax2.legend()

        if frame == frames - 1:
            ax1.set_title("Creation Complete: 'The Concept of 8'", color='green', fontweight='bold')
            ax1.text(0, 9, "Ideal = Reality - Noise", fontsize=12, color='green')

    ani = FuncAnimation(fig, update, frames=frames, interval=100)
    return HTML(ani.to_jshtml())

run_md_creation_demo()

⑭MD理論による創造：文脈による意味の分岐

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from scipy.ndimage import gaussian_filter, laplace
from IPython.display import HTML

def run_md_genesis():
    # --- 世界の設定 ---
    size = 64
    frames = 60

    # 1. 創造の種 (The Seed of 0->1)
    # 完全にランダムなノイズ。ここには何の意味もない。
    np.random.seed(42) # 同じ種を使う
    seed_noise = np.random.uniform(0, 1, (size, size))

    # 世界A: 生物的文脈 (Context: Organic)
    # G_bio = 「滑らかで、丸みを帯び、膜を持つもの」
    world_bio = seed_noise.copy()

    # 世界B: 機械的文脈 (Context: Mechanical)
    # G_mech = 「直角的で、鋭利で、グリッドに沿うもの」
    world_mech = seed_noise.copy()

    history_bio = []
    history_mech = []

    print("MD Genesis: Creating Meaning from Chaos...")

    # --- MD Logic Functions ---

    def apply_organic_law(img):
        # 生物の法則 G_bio:
        # 「急激な変化（カド）はノイズである」 -> 平滑化
        # 「生命は拡散と凝集を繰り返す」 -> 反応拡散的な処理

        # 1. 理想(Smoothness)との差分(Delta)を計算
        smooth = gaussian_filter(img, sigma=1.0)
        delta = img - smooth

        # 2. 引き算 (ノイズ除去)
        # カドを削る
        img = img - delta * 0.1

        # 3. 創造的ゆらぎ (Life Force)
        # 活性化(Sigmoid)してコントラストを強める（細胞膜の形成）
        img = 1 / (1 + np.exp(-(img - 0.5) * 10))
        return img

    def apply_mechanical_law(img):
        # 機械の法則 G_mech:
        # 「曖昧な中間色はノイズである」 -> 2値化
        # 「曲線はノイズである」 -> グリッドへの強制

        # 1. 理想(Grid)との差分(Delta)
        # 縦横のラインを強調
        grad_x = np.abs(np.roll(img, 1, axis=0) - img)
        grad_y = np.abs(np.roll(img, 1, axis=1) - img)
        structure = grad_x + grad_y

        # 2. 引き算 (曖昧さの除去)
        # 構造的でない部分を削る
        delta = img - (img > 0.5).astype(float) # 0か1以外はノイズ
        img = img - delta * 0.05

        # 3. 創造的拘束 (Grid Lock)
        # 閾値処理でパキッとさせる
        img = np.clip(img + structure * 0.1, 0, 1)
        return img

    # --- Simulation Loop ---
    for i in range(frames):
        # Context A: Organic Evolution
        world_bio = apply_organic_law(world_bio)
        # ノイズ注入（突然変異）
        if i < 20: world_bio += np.random.normal(0, 0.05, (size, size))
        history_bio.append(world_bio.copy())

        # Context B: Mechanical Assembly
        world_mech = apply_mechanical_law(world_mech)
        # ノイズ注入（設計変更）
        if i < 20: world_mech += np.random.normal(0, 0.05, (size, size))
        history_mech.append(world_mech.copy())

    # --- Visualization ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    def update(frame):
        ax1.clear()
        ax2.clear()

        # Left: Organic
        ax1.imshow(history_bio[frame], cmap='magma', vmin=0, vmax=1)
        ax1.set_title(f"Context: ORGANIC (Life)\nCreating 'Cell' from Noise", color='purple', fontweight='bold')
        ax1.axis('off')

        # Right: Mechanical
        ax2.imshow(history_mech[frame], cmap='viridis', vmin=0, vmax=1)
        ax2.set_title(f"Context: MECHANICAL (Tech)\nCreating 'Chip' from Noise", color='green', fontweight='bold')
        ax2.axis('off')

        if frame == 0:
            fig.suptitle("Input: Same Random Noise ($\Psi$)", fontsize=16)
        elif frame == frames - 1:
            fig.suptitle("Output: Different 'Meaning' via Context ($G$)", fontsize=16, fontweight='bold')

    ani = FuncAnimation(fig, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_md_genesis()

⑮完全犯罪を暴く：アクティブ・センシング（Active MD）・能動的知覚

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from IPython.display import HTML

def run_active_sensing():
    # --- 世界の設定 ---
    frames = 100
    x = np.linspace(0, 10, 200)

    # 1. G (背景): 穏やかな波
    def get_background(t):
        return np.sin(x - t * 0.1) * 0.5

    # 2. Enemy (擬態): 背景に合わせて動くステルス迷彩
    # x=5.0 の位置に潜んでいる
    enemy_pos_idx = 100 # x[100] approx 5.0
    enemy_width = 10

    # --- モード設定 ---
    # 0-50フレーム: Passive Mode (ただ見ているだけ)
    # 50-100フレーム: Active Mode (干渉波を打つ！)

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))

    print("Initiating Active Sonar Test...")

    def update(frame):
        t = frame

        # --- Reality Creation ---
        # 背景
        bg = get_background(t)

        # ステルス敵のロジック:
        # 「背景と同じ値」を出力して姿を消す (Perfect Mimicry)
        reality = bg.copy()

        # 敵がいる場所
        enemy_mask = np.zeros_like(x)
        enemy_mask[enemy_pos_idx-enemy_width : enemy_pos_idx+enemy_width] = 1.0

        # --- Active Ping Logic (The Trick) ---
        ping = np.zeros_like(x)
        is_active = frame > 50

        if is_active:
            # 50フレーム以降、小さな衝撃波(Ping)を加える
            # Pingは物理法則に従って伝播する
            ping_t = frame - 50
            ping = np.exp(-((x - (ping_t * 0.2))**2) * 10) * 0.3

            # 【重要】現実世界では、Pingが背景に乗る
            reality += ping

            # 【ここが敵のボロ】
            # 敵は「背景(G)」には擬態できているが、「突発的なPing」には
            # 反応速度が追いつかず、擬態が剥がれる（あるいは不自然に反応しない）
            # ここでは「Pingの影響を受けずに背景のふりをしてしまう」というボロを再現
            reality[enemy_mask > 0] -= ping[enemy_mask > 0] * 0.8 # Pingを無視してしまう（不自然）

        # --- MD Detection Logic ---
        # 予測 (Prediction) P
        # Passive時: P = Background
        # Active時:  P = Background + Expected_Ping_Response
        expected_reality = bg + ping

        # 差分 (Delta)
        delta = np.abs(reality - expected_reality)

        # --- Visualization ---
        ax1.clear()
        ax2.clear()

        # Visual 1: Reality View
        ax1.set_ylim(-1.5, 1.5)
        ax1.plot(x, reality, 'k-', linewidth=1, label='Reality ($\Psi$)')
        ax1.plot(x, bg, 'g--', alpha=0.3, label='Normal G')

        # 敵の位置を可視化（本来は見えない）
        ax1.fill_between(x, -1.5, 1.5, where=enemy_mask>0, color='red', alpha=0.1, label='Hidden Enemy')

        mode_text = "PASSIVE MODE (Observing)"
        if is_active:
            mode_text = "ACTIVE MODE (Pinging)"
            ax1.plot(x, ping, 'b:', label='Active Ping')

        ax1.set_title(f"World View: {mode_text}", fontweight='bold')
        ax1.legend(loc='upper right', fontsize=8)

        # Visual 2: MD Detection (Delta)
        ax2.set_ylim(0, 1.0)
        ax2.plot(x, delta, 'r-', linewidth=2, label='Detected Anomaly ($\Delta$)')
        ax2.fill_between(x, 0, delta, color='red', alpha=0.3)

        # 判定
        max_delta = np.max(delta)
        status = "CLEAR"
        if max_delta > 0.1:
            status = "DETECTED!"
            ax2.text(5, 0.5, "MIMICRY EXPOSED!", color='red', fontweight='bold', ha='center')

        ax2.set_title(f"MD Theory Analysis: {status}")
        ax2.legend(loc='upper right')

    ani = FuncAnimation(fig, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_active_sensing()

⑯MD共鳴実験：二つの神の対話 (Mutual Interference)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.ndimage import gaussian_filter
from IPython.display import HTML

def run_md_resonance():
    # --- 世界の設定 ---
    # 共有された1次元の世界（通信回線、あるいは量子場）
    width = 200
    frames = 200

    # 世界の初期状態（無）
    world_state = np.zeros(width)

    # 履歴記録用
    history_world = []

    # 二つのMDエージェントの設定
    # Agent A: "秩序"の神 (G = Smooth Wave)
    # Agent B: "混沌"の神 (G = Sharp Spike)

    print("Initiating Mutual Interference Protocol...")

    def agent_a_logic(current_world):
        # Aの理想: 「世界は滑らかであるべきだ」
        # G_a = ローパスフィルタを通した世界
        ideal = gaussian_filter(current_world, sigma=3.0)

        # 差分検知 (Delta)
        delta = current_world - ideal

        # 引き算 (Subtract Delta)
        # ノイズ（ギザギザ）を削る
        # 反応率 0.2
        action = -delta * 0.2

        # 【能動的干渉 (Ping)】
        # 世界があまりに静かすぎると、不安になって「問い」を投げる
        if np.std(current_world) < 0.01:
            # 長波長のPingを投げる
            x = np.linspace(0, 4*np.pi, len(current_world))
            ping = np.sin(x) * 0.5
            action += ping

        return action

    def agent_b_logic(current_world):
        # Bの理想: 「世界は刺激的であるべきだ」
        # G_b = ハイパスフィルタ的な世界（変化点のみ）
        # 自分の値を極端に保とうとする
        ideal = np.sign(current_world) * np.abs(current_world) ** 0.5

        # Bは「滑らかさ」を敵（ノイズ）とみなす
        smooth = gaussian_filter(current_world, sigma=3.0)
        # BにとってのDelta = 滑らかな成分
        delta = smooth

        # 引き算 (Subtract Smoothness) -> 結果的にギザギザが残る
        # 反応率 0.2
        action = -delta * 0.2

        # 【能動的干渉 (Ping)】
        # 世界が静かすぎると、鋭いスパイク（問い）を打ち込む
        if np.std(current_world) < 0.01:
            idx = np.random.randint(0, len(current_world))
            action[idx] += 2.0 # Impulse Ping

        return action

    # --- Simulation Loop ---
    for i in range(frames):
        # 1. 相互観測
        # AとBは同時に今の世界を見る
        obs = world_state.copy()

        # 2. 意思決定 (MD Subtraction)
        act_a = agent_a_logic(obs)
        act_b = agent_b_logic(obs)

        # 3. 世界への干渉 (Interaction)
        # 世界は二人の神の「引き算の合成」によって更新される
        # World_new = World_old + Action_A + Action_B
        # 物理的な減衰（Entropy）も少し入れる
        world_state = world_state * 0.95 + act_a + act_b

        history_world.append(world_state.copy())

    # --- Visualization ---
    fig, ax = plt.subplots(figsize=(10, 6))

    # データを画像として表示（横軸: 空間, 縦軸: 時間）
    # これにより「パターン（言語）」の発生を可視化する
    history_array = np.array(history_world)

    im = ax.imshow(history_array, aspect='auto', cmap='seismic', vmin=-1, vmax=1)
    ax.set_title("MD Resonance: Emergence of Pattern from Conflict", fontweight='bold')
    ax.set_xlabel("Space (X)")
    ax.set_ylabel("Time (Frame)")
    plt.colorbar(im, label="Signal Intensity")

    # アニメーション用には、この「履歴画像」が徐々に描かれる様子を見せるのではなく
    # この時空図（Space-Time Plot）自体が結果そのものなので、静止画+解説で十分だが
    # 動きを見るために断面図のアニメーションを作成する

    fig2, ax2 = plt.subplots(figsize=(10, 4))
    line, = ax2.plot([], [], 'k-', linewidth=1.5)
    ax2.set_ylim(-2, 2)
    ax2.set_xlim(0, width)
    ax2.set_title("Real-time Interaction (A vs B)")

    def update(frame):
        current = history_world[frame]
        line.set_data(range(width), current)
        return line,

    ani = FuncAnimation(fig2, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_md_resonance()

⑰MD連鎖実験：共鳴による「翻訳」

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_translation():
    # --- 1. 学習フェーズ (Learning the Correlation) ---
    # 辞書を覚えるのではなく、「関係性（相関）」だけを統計的に記録する

    dim = 50
    n_samples = 1000

    # パターンA: "Sine Shape" <-> "High Freq Sound"
    t = np.linspace(0, 4*np.pi, dim)
    shape_A = np.sin(t)
    sound_A = np.sin(t * 3) # Fast

    # パターンB: "Square Shape" <-> "Low Freq Sound"
    shape_B = np.sign(np.sin(t))
    sound_B = np.sin(t * 1) # Slow

    # 結合データの作成 (Joint Vector)
    # [Shape, Sound] という一つの長いベクトルとして扱う
    X_train = []
    for _ in range(n_samples):
        if np.random.rand() > 0.5:
            # パターンAにノイズを乗せたもの
            s = shape_A + np.random.normal(0, 0.1, dim)
            a = sound_A + np.random.normal(0, 0.1, dim)
        else:
            # パターンBにノイズを乗せたもの
            s = shape_B + np.random.normal(0, 0.1, dim)
            a = sound_B + np.random.normal(0, 0.1, dim)
        X_train.append(np.concatenate([s, a]))

    X_train = np.array(X_train)

    # G (Joint Law) の獲得
    # ここには「形」と「音」の共分散（つながり）が記録される
    g_mean = np.mean(X_train, axis=0)
    g_cov = np.cov(X_train, rowvar=False)
    # 逆行列 (Precision Matrix) - これが「翻訳回路」になる
    g_inv_cov = np.linalg.inv(g_cov + np.eye(len(g_cov)) * 0.01)

    # --- 2. 翻訳テスト (Translation Task) ---
    # 入力: "Square Shape" (明確) + "Noise" (無意味)
    # 期待: ノイズが削られて "Low Freq Sound" になるはず

    input_shape = shape_B + np.random.normal(0, 0.05, dim) # 正しい形
    input_sound = np.random.normal(0, 2.0, dim) # 完全なノイズ（最初はデタラメ）

    current_state = np.concatenate([input_shape, input_sound])

    history = []
    frames = 60

    print("MD Rosetta: Translating Shape to Sound via Resonance...")

    # MD連鎖ロジック (Iterative Subtraction)
    # 全体のマハラノビス距離を最小化するように、
    # ノイズ部分（Sound）を修正していく

    learning_rate = 0.5

    for i in range(frames):
        history.append(current_state.copy())

        # 1. 違和感(Delta)の計算
        # Joint Vector としての偏差
        diff = current_state - g_mean

        # 2. 勾配の計算 (マハラノビス距離の微分)
        # 距離 D^2 = diff.T * inv_cov * diff
        # dD^2/d(diff) = 2 * inv_cov * diff
        gradient = np.dot(g_inv_cov, diff)

        # 3. 引き算 (Translation)
        # ただし、入力側(Shape)は「観測された事実」なので動かさない（固定）
        # 出力側(Sound)だけを動かす
        update = -gradient * learning_rate

        # Shape部分は固定(Masking)
        update[:dim] = 0

        current_state += update

        # 少しノイズを入れて探索させる
        if i < 20: current_state[dim:] += np.random.normal(0, 0.1, dim)

    # --- Visualization ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    def update(frame):
        ax1.clear()
        ax2.clear()

        state = history[frame]
        vis_shape = state[:dim]
        vis_sound = state[dim:]

        # Left: Input (Visual World)
        ax1.plot(t, vis_shape, 'b-', linewidth=2, label='Input: Shape')
        ax1.set_title("Input Modality (Visual)\nFixed Fact", color='blue', fontweight='bold')
        ax1.set_ylim(-2, 2)
        ax1.legend(loc='upper right')

        # Right: Output (Auditory World)
        ax2.plot(t, vis_sound, 'r-', linewidth=2, label='Output: Sound')
        # 正解（Low Freq）を薄く表示
        ax2.plot(t, sound_B, 'g--', alpha=0.3, label='Target Meaning')

        status = "Translating..."
        if frame == frames - 1: status = "Translation Complete"

        ax2.set_title(f"Output Modality (Auditory)\n{status}", color='red', fontweight='bold')
        ax2.set_ylim(-2, 2)
        ax2.legend(loc='upper right')

        fig.suptitle(f"MD Translation: Frame {frame}", fontsize=14)

    ani = FuncAnimation(fig, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_md_translation()

⑱MD法則発見器：未知の定理を導き出す

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_law_discovery():
    # --- 1. 世界の創造 (Hidden Law) ---
    # 人間には分からない複雑な法則をランダムに作る
    # 法則の形: a*X^2 + b*Y^2 + c*X*Y + d*Z + e = 0
    # これは2次曲面（放物面や双曲面など）の方程式

    np.random.seed(42)
    # 秘密の係数 (True Parameters)
    true_params = np.random.uniform(-5, 5, 5) # [a, b, c, d, e]
    # d (Zの係数) は0にならないようにする（Z = ... の形にするため）
    if abs(true_params[3]) < 1: true_params[3] = 1.0

    # 文字列での数式（正解）
    def format_eq(p):
        return f"{p[0]:.2f}x² + {p[1]:.2f}y² + {p[2]:.2f}xy + {p[3]:.2f}z + {p[4]:.2f} = 0"

    true_eq_str = format_eq(true_params)
    print(f"SECRET LAW (God's Truth): {true_eq_str}")

    # --- 2. 観測データ生成 (Noisy Reality) ---
    # この法則に従うデータを作るが、激しいノイズを乗せる
    n_samples = 500

    x = np.random.uniform(-10, 10, n_samples)
    y = np.random.uniform(-10, 10, n_samples)

    # Zを逆算: d*Z = -(aX^2 + bY^2 + cXY + e)
    # Z = -(aX^2 + bY^2 + cXY + e) / d
    z_true = -(true_params[0]*x**2 + true_params[1]*y**2 + true_params[2]*x*y + true_params[4]) / true_params[3]

    # 観測データ (Psi) = 真実 + ノイズ
    # ノイズが大きすぎると、人間の目にはただの「煙」にしか見えない
    noise_level = 5.0
    z_obs = z_true + np.random.normal(0, noise_level, n_samples)

    # --- 3. MD理論による法則発見 (The Discovery) ---
    # MD理論のアプローチ:
    # 「法則とは、分散がゼロになる方向（特異点）のことである」
    # 我々は、変数 [x^2, y^2, xy, z, 1] の結合空間における
    # 「最もマハラノビス距離が短い（＝変化しない）ベクトル」を探す。
    # これが「最小固有値法」あるいは「全最小二乗法」と呼ばれるMDの奥義。

    # 特徴量ベクトルの構築 (Basis Functions)
    # Psi = [x^2, y^2, xy, z, 1]
    Psi = np.column_stack([x**2, y**2, x*y, z_obs, np.ones(n_samples)])

    # 共分散行列 (Covariance Matrix) の計算
    # これが世界の「相関構造」のすべて
    G_cov = np.cov(Psi, rowvar=False)

    # 固有値分解 (Eigen Decomposition)
    # 固有値が最小になる固有ベクトル = 分散が最も小さい方向 = 「法則」
    eigenvalues, eigenvectors = np.linalg.eigh(G_cov)

    # 最小固有値に対応するベクトルを取り出す
    min_idx = np.argmin(eigenvalues)
    discovered_params = eigenvectors[:, min_idx]

    # 係数のスケール合わせ (Zの係数などを基準に正規化して比較可能にする)
    # 真のパラメータのノルムに合わせる（方向が重要なので）
    scale_factor = np.linalg.norm(true_params) / np.linalg.norm(discovered_params)
    # 符号も合わせる
    if np.sign(discovered_params[3]) != np.sign(true_params[3]):
        scale_factor *= -1

    md_params = discovered_params * scale_factor
    md_eq_str = format_eq(md_params)

    print(f"MD DISCOVERY:             {md_eq_str}")

    # 誤差率
    error = np.mean(np.abs(true_params - md_params))
    print(f"Coefficient Error: {error:.4f}")

    # --- 4. Visualization ---
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # グリッド生成（曲面描画用）
    gx = np.linspace(-10, 10, 20)
    gy = np.linspace(-10, 10, 20)
    GX, GY = np.meshgrid(gx, gy)

    # MDが発見した法則に基づいてZを計算
    # p0*x^2 + p1*y^2 + p2*xy + p3*z + p4 = 0
    # z = -(p0*x^2 + p1*y^2 + p2*xy + p4) / p3
    GZ_md = -(md_params[0]*GX**2 + md_params[1]*GY**2 + md_params[2]*GX*GY + md_params[4]) / md_params[3]

    # アニメーション設定
    ax.set_title("MD Discovery: Extracting Hidden Law from Chaos", fontweight='bold')
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    ax.set_zlabel("Z")

    # 初期視点
    ax.view_init(elev=20, azim=0)

    def update(frame):
        ax.clear()
        ax.set_title(f"MD Discovery: Extracting Hidden Law from Chaos\nTrue Law: {true_eq_str}\nMD Found: {md_eq_str}", fontsize=10)
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        ax.set_zlim(np.min(z_obs), np.max(z_obs))

        # 1. 観測データ (Noise) - 半透明の点
        ax.scatter(x, y, z_obs, c='gray', alpha=0.3, s=5, label='Noisy Data ($\Psi$)')

        # 2. 発見された法則 (Surface) - ワイヤーフレーム
        # 回転させる
        ax.view_init(elev=20 + np.sin(frame/20)*10, azim=frame)

        ax.plot_wireframe(GX, GY, GZ_md, color='red', alpha=0.6, rstride=2, cstride=2, label='Discovered Law ($G$)')

        # 凡例は最初だけ
        if frame == 0: ax.legend()

    ani = FuncAnimation(fig, update, frames=120, interval=50)
    return HTML(ani.to_jshtml())

run_md_law_discovery()

⑲MD・ジェネシス：確率的創造（Stochastic Creation）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.ndimage import convolve
from IPython.display import HTML

def run_md_world_creation():
    # --- 世界の設定 ---
    width = 100
    height = 100
    frames = 100

    # 最初は「無（Void）」
    # world_state = np.zeros((height, width))
    # いや、ビッグバン（初期ノイズ）を起こそう
    np.random.seed(999)
    world_state = np.random.rand(height, width)

    # 履歴
    history = []

    # --- 神のパラメータ (The Preferences of God) ---
    # ここを変えると「好きな現実」が作れる
    # パターン形成（チューリング・パターン）のカーネルをMD的に定義

    # 設定A: 迷宮の世界 (Labyrinth)
    # 近傍は仲間(1.0)、少し遠くは敵(-0.5)
    kernel_labyrinth = np.array([[0, -1, 0],
                                 [-1, 4, -1],
                                 [0, -1, 0]]) * 0.1

    # 設定B: 群島の世界 (Islands)
    # 広範囲の抑制を強くする
    kernel_islands = np.ones((5,5)) * -0.1
    kernel_islands[2,2] = 1.0

    # 今回は「迷宮」を作る
    current_kernel = kernel_labyrinth

    print("Initiating World Construction...")

    def apply_gods_will(world, kernel):
        # 1. 拡散 (Diffusion)
        # 物理法則G: 「隣り合うものは影響し合う」
        # これを畳み込みで計算
        interaction = convolve(world, kernel, mode='wrap')

        # 2. 成長と淘汰 (MD Selection)
        # MD理論: 「法則(G)に従うものは強化し、従わないものは抑制せよ」
        # interaction > 0 なら法則に合致（強化）、 < 0 なら不適合（抑制）

        # 成長速度
        growth_rate = 0.1

        # 世界の更新
        # World_new = World_old + (Interaction * Rate)
        new_world = world + interaction * growth_rate

        # 3. 揺らぎの注入 (Random Injection)
        # 常に新しい可能性(Noise)を少しだけ入れる
        # これがないと世界は死ぬ（固定化する）
        mutation = (np.random.rand(*world.shape) - 0.5) * 0.05
        new_world += mutation

        # 4. 物理的制約 (Activation Function)
        # 世界の物質量は 0.0 ~ 1.0 に収まる（資源の限界）
        # Sigmoid的なクリッピング
        new_world = np.clip(new_world, 0, 1)

        return new_world

    # --- Simulation Loop ---
    for i in range(frames):
        # 世界を更新
        world_state = apply_gods_will(world_state, current_kernel)

        # [神の気まぐれ]
        # 途中(50フレーム目)で「物理法則」を変えてみる
        # 迷宮 -> 群島
        if i == 50:
            current_kernel = kernel_islands
            # 突然変異を大量投下（大災害）
            world_state += np.random.rand(height, width) * 0.5

        history.append(world_state.copy())

    # --- Visualization ---
    fig, ax = plt.subplots(figsize=(8, 8))

    im = ax.imshow(history[0], cmap='inferno', vmin=0, vmax=1)
    ax.axis('off')

    # タイトル更新用
    title_text = ax.text(0.5, 1.05, "", transform=ax.transAxes, ha="center", fontweight="bold", fontsize=12)

    def update(frame):
        im.set_data(history[frame])

        phase = "Phase 1: Age of Labyrinth (Order)"
        if frame >= 50:
            phase = "Phase 2: Age of Islands (Catastrophe & Rebirth)"

        title_text.set_text(f"World Creation: Frame {frame}\n{phase}")
        return [im, title_text]

    ani = FuncAnimation(fig, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_md_world_creation()

⑳MD・サイエンティスト：未知の定理の自動発見

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_scientific_discovery():
    # --- 1. 変数のスープ (Candidate Library) ---
    # 科学者が実験で計測しそうなあらゆる変数を準備する
    n_samples = 1000

    # 独立変数 (Inputs)
    x = np.random.uniform(-3, 3, n_samples)
    y = np.random.uniform(-3, 3, n_samples)

    # 候補となる項 (Terms)
    # ここから「未知の法則」が構成される
    terms = {
        'x': x,
        'y': y,
        'x^2': x**2,
        'y^2': y**2,
        'sin(x)': np.sin(x),
        'cos(y)': np.cos(y),
        'x*y': x*y,
        '1': np.ones(n_samples) # 定数項
    }

    term_names = list(terms.keys())
    # データ行列を作る [Samples, Features]
    # ここに「未知の従属変数 z」も後で加える

    # --- 2. 未知の法則の生成 (God's Dice) ---
    # ランダムに係数を決めて、新しい物理法則を作る
    # 例: 3.5*x^2 - 2.1*sin(x) + 0.5*x*y + z - 5 = 0

    np.random.seed(None) # 毎回違う法則を作る

    # ランダムな係数 (-5 ~ 5)
    coeffs = np.random.uniform(-5, 5, len(term_names))
    # スパース性を持たせる（いくつかの項は使わない＝0にする）
    # これにより「シンプルで美しい数式」を目指す
    mask = np.random.rand(len(term_names)) > 0.5
    if not any(mask): mask[0] = True # 少なくとも1つは使う
    coeffs *= mask

    # Z (観測対象) を逆算する
    # Law: Sum(coeffs * terms) + z = 0  =>  z = -Sum(...)
    z_true = np.zeros(n_samples)
    formula_parts = []

    for i, name in enumerate(term_names):
        val = terms[name]
        c = coeffs[i]
        if c != 0:
            z_true -= c * val
            formula_parts.append(f"{c:+.2f}·{name}")

    # 真の法則（文字列）
    true_law_str = "z " + "".join(formula_parts) + " = 0"
    print(f"UNKNOWN LAW GENERATED: {true_law_str}")

    # --- 3. 観測 (Observation with Noise) ---
    # 現実はノイズまみれ
    z_obs = z_true + np.random.normal(0, 0.5, n_samples)

    # --- 4. MD Discovery (Dimensionality Reduction) ---
    # MD理論: 「法則とは、多次元空間における分散が最小の方向である」

    # 全ての変数を結合した行列を作る
    # Data = [x, y, x^2, ..., z]
    data_matrix = []
    for name in term_names:
        data_matrix.append(terms[name])
    data_matrix.append(z_obs) # 最後にZを追加

    X = np.column_stack(data_matrix)
    feature_names = term_names + ['z']

    # 共分散行列
    cov_mat = np.cov(X, rowvar=False)

    # 固有値分解
    evals, evecs = np.linalg.eigh(cov_mat)

    # 最小固有値に対応する固有ベクトルが「法則の係数」
    min_idx = np.argmin(evals)
    law_vector = evecs[:, min_idx]

    # 係数の正規化 (Zの係数を1にする)
    z_coeff_idx = len(feature_names) - 1
    scale = 1.0 / law_vector[z_coeff_idx]
    law_vector *= scale

    # 発見された数式の構築
    discovered_parts = []
    for i, name in enumerate(feature_names):
        c = law_vector[i]
        # 小さすぎる係数は0とみなす（ノイズ除去）
        if abs(c) > 0.1:
            discovered_parts.append(f"{c:+.2f}·{name}")

    discovered_law_str = "".join(discovered_parts) + " = 0"
    print(f"MD DISCOVERY        : {discovered_law_str}")

    # --- 5. Visualization ---
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # データを間引いて表示
    idx = np.random.choice(n_samples, 200, replace=False)

    # 軸の選択（可視化のため、影響力の大きい2つの変数を選ぶ）
    # 係数の絶対値が大きい順にソート
    coef_mags = np.abs(law_vector[:-1]) # Z以外
    sorted_indices = np.argsort(coef_mags)[::-1]

    idx_x = sorted_indices[0]
    idx_y = sorted_indices[1] if len(sorted_indices) > 1 else 0

    name_x = feature_names[idx_x]
    name_y = feature_names[idx_y]

    # 3D Plot
    # X軸: 最も支配的な項1, Y軸: 最も支配的な項2, Z軸: 観測値Z
    vx = X[idx, idx_x]
    vy = X[idx, idx_y]
    vz = z_obs[idx]

    # 予測平面（MD Law）
    # グリッド作成
    gx = np.linspace(min(vx), max(vx), 20)
    gy = np.linspace(min(vy), max(vy), 20)
    GX, GY = np.meshgrid(gx, gy)

    # 可視化のために簡易的にZを逆算 (他の項は平均値と仮定)
    # これ厳密な可視化は難しいので、あくまで「適合している感」を出す
    # z = - (c1*v1 + c2*v2 + const)
    c1 = law_vector[idx_x]
    c2 = law_vector[idx_y]
    const_val = law_vector[len(law_vector)-2] # 定数項('1')の係数

    GZ = -(c1*GX + c2*GY + const_val)

    ax.set_title(f"Discovery of New Law\nTarget: {name_x} & {name_y} -> z", fontsize=10)
    ax.set_xlabel(name_x)
    ax.set_ylabel(name_y)
    ax.set_zlabel("z")

    def update(frame):
        ax.clear()
        ax.set_title(f"Discovery of New Law\nTruth: {true_law_str}\nFound: {discovered_law_str}", fontsize=9)
        ax.set_xlabel(name_x)
        ax.set_ylabel(name_y)
        ax.set_zlabel("z")

        # データ点
        ax.scatter(vx, vy, vz, c='blue', alpha=0.3, label='Observed Data')

        # 発見された法則の面
        ax.plot_wireframe(GX, GY, GZ, color='red', alpha=0.5, rstride=2, cstride=2, label='MD Law Surface')

        # 視点回転
        ax.view_init(elev=20, azim=frame)
        if frame == 0: ax.legend()

    ani = FuncAnimation(fig, update, frames=120, interval=50)
    return HTML(ani.to_jshtml())

run_md_scientific_discovery()

２１。理論的創発：統計的一般相対性理論 (Statistical General Relativity)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_theoretical_genesis():
    # --- 世界の設定 ---
    # 2次元の概念空間
    x_range = np.linspace(-5, 5, 20)
    y_range = np.linspace(-5, 5, 20)
    X, Y = np.meshgrid(x_range, y_range)

    # 1. 情報の質量 (Semantic Mass)
    # ここに「強い概念（例：『王様』）」があるとする
    # データが密集している＝分散が小さい＝重力が強い
    mass_center = np.array([0, 0])
    mass_cov = np.array([[0.5, 0], [0, 0.5]]) # 非常に高密度

    # 2. 古典的MDの計量 (Global Metric)
    # 宇宙全体で一定の共分散を持つ（平坦な時空）
    global_cov = np.array([[3.0, 0], [0, 3.0]])
    global_inv = np.linalg.inv(global_cov)

    # --- The Invention: Gravitational Metric Tensor ---
    # 新理論: 共分散行列は場所によって変わる！
    # G(x) = Global_G + Local_Gravity(x)

    def get_metric_tensor(pos):
        # 位置 pos における「距離の縮尺」を計算する
        # 質点に近いほど、空間が圧縮される（＝距離が短くなる＝引き寄せられる）

        # 質点からの距離
        diff = pos - mass_center
        dist_sq = np.dot(diff, diff)

        # 重力ポテンシャル (距離の二乗に反比例して空間を歪める)
        # 密度が高い場所ほど、情報量としての距離は「近く」なるべき
        # 重力係数
        gravity = 5.0 * np.exp(-dist_sq / 2.0)

        # 局所的な計量テンソル (Metric Tensor)
        # G_local = G_global * (1 - gravity_effect)
        # 重力が強いと係数が小さくなる -> 同じ座標差でも距離Dが小さくなる -> 近道になる
        scaling = 1.0 / (1.0 + gravity)

        local_inv_cov = global_inv * scaling
        return local_inv_cov

    # --- 測地線 (Geodesic) の計算 ---
    # 2点間の「本当の最短ルート」を探す
    # Euler-Lagrange方程式を解く（変分原理）

    # スタート地点（概念A）とゴール地点（概念B）
    start_pos = np.array([-4, -2])
    end_pos = np.array([4, 2])

    # A. 古典的MDのパス（直線）
    t_vals = np.linspace(0, 1, 100)
    classic_path = start_pos + np.outer(t_vals, (end_pos - start_pos))

    # B. 重力的MDのパス（曲線）
    # 簡易的に、グリッド上のコスト探索（ダイクストラ的な）で近似する
    # あるいは、重力に引かれる粒子の軌道としてシミュレート

    def geodesic_derivative(state, t):
        # 物理シミュレーション:
        # 「情報の等価性」を保とうとする力が働く
        pos = state[:2]
        vel = state[2:]

        # 勾配（重力）を計算
        diff = mass_center - pos
        dist = np.linalg.norm(diff)
        force = diff / (dist**3 + 0.1) * 2.0 # 引力

        # 減衰（ゴールに向かう意志とのバランス）
        target_dir = end_pos - pos
        target_force = target_dir * 0.5

        acc = force + target_force - vel * 0.5
        return np.concatenate([vel, acc])

    # 初期速度（ゴールへ向かう）
    init_vel = (end_pos - start_pos) * 0.5
    state0 = np.concatenate([start_pos, init_vel])
    t_sim = np.linspace(0, 10, 100)

    solution = odeint(geodesic_derivative, state0, t_sim)
    grav_path = solution[:, :2]

    # ゴール付近でクリップ
    dist_to_end = np.linalg.norm(grav_path - end_pos, axis=1)
    min_idx = np.argmin(dist_to_end)
    grav_path = grav_path[:min_idx+1]

    # --- Visualization ---
    fig, ax = plt.subplots(figsize=(10, 8))

    # 重力場（ポテンシャル）の可視化
    Z = np.zeros_like(X)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            p = np.array([X[i,j], Y[i,j]])
            # 計量テンソルの行列式（空間の密度）
            mt = get_metric_tensor(p)
            Z[i,j] = np.linalg.det(mt)

    # 等高線
    ax.contourf(X, Y, -Z, cmap='Blues', levels=20, alpha=0.3)
    ax.scatter(mass_center[0], mass_center[1], c='black', s=200, label='Semantic Mass (Concept Core)')

    # アニメーション用
    line_classic, = ax.plot([], [], 'g--', linewidth=2, label='Classical MD (Euclidean/Flat)')
    line_grav, = ax.plot([], [], 'r-', linewidth=3, label='Gravitational MD (Curved Space)')
    point_grav, = ax.plot([], [], 'ro', markersize=8)

    ax.set_title("Theoretical Genesis: Statistical General Relativity\nCurvature of Concept Space via Data Density", fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

    def update(frame):
        # Classical: Just linear interpolation
        idx_c = min(frame, len(classic_path)-1)
        line_classic.set_data(classic_path[:idx_c+1, 0], classic_path[:idx_c+1, 1])

        # Gravitational: Curved path
        idx_g = min(frame, len(grav_path)-1)
        line_grav.set_data(grav_path[:idx_g+1, 0], grav_path[:idx_g+1, 1])
        point_grav.set_data([grav_path[idx_g, 0]], [grav_path[idx_g, 1]])

        return line_classic, line_grav, point_grav

    ani = FuncAnimation(fig, update, frames=100, interval=50)
    return HTML(ani.to_jshtml())

run_md_theoretical_genesis()

２２．MD・メタフィジクス：新定理の自動証明

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_new_theorem_discovery():
    # --- 1. 世界の創造 (Genesis of Alien Universe) ---
    # 人類が知らない物理法則を持つ宇宙を作る
    n_samples = 2000

    # 観測可能な変数 (Observables)
    # S: 空間の歪み (Space Curvature)
    # T: 時間の密度 (Time Density)
    S = np.random.uniform(-2, 2, n_samples)
    T = np.random.uniform(0, 5, n_samples)

    # --- 2. 存在しない二つの定理 (The Non-Existent Theorems) ---

    # 定理A: 意志の流体 (Will)
    # "意志は、空間の歪みの二乗に比例し、時間の密度と共鳴する"
    # W = 3.0 * S^2 + 1.5 * sin(T)
    Will = 3.0 * S**2 + 1.5 * np.sin(T)

    # 定理B: 運命の粘性 (Fate)
    # "運命は、空間の歪みの逆相関であり、時間の密度の指数関数で減衰する"
    # F = -2.0 * S * T + 0.5 * exp(T/5)
    Fate = -2.0 * S * T + 0.5 * np.exp(T/5)

    # --- 3. 未知の大統一 (The Hidden Unified Law) ---
    # ここに「意志」と「運命」を繋ぐ、さらに高次の法則を隠す
    # 人間にはこの関係性が見えないとする
    # Law: alpha * Will + beta * Fate + gamma * (S*Will) + delta * (T*Fate) = Constant
    # 非線形で複雑な相互作用項を入れる

    # 定義する真の法則 (God's Equation):
    # 2.0 * Will + 3.0 * Fate + 4.0 * (S * T) - 10.0 = 0
    # (※ S*T は相互作用項)

    # この法則に従うように、観測データに「補正項 (Dark Matter)」を加える
    # ここでは単純化のため、観測された Will と Fate が
    # 以下の「超法則」によって縛られている世界をシミュレートする

    # 項目の準備
    term_will = Will
    term_fate = Fate
    term_interaction = S * T # 空間と時間の交差
    term_const = np.ones(n_samples)

    # 真の係数 (True Coefficients)
    c_will = 2.0
    c_fate = 3.0
    c_inter = 6.0 # WillとFateの媒介項 (ここが肝)

    # ここで矛盾が生じないように、Z (未知のエネルギー) を逆算してデータに含める
    # Law: 2*Will + 3*Fate + 6*(S*T) + Z = 0
    # -> Z = -(2*Will + 3*Fate + 6*S*T)
    Unknown_Energy = -(c_will * term_will + c_fate * term_fate + c_inter * term_interaction)

    # --- 4. MDによる新定理の導出 (Deriving the Theorem) ---

    print("Initiating MD Metaphysics Engine...")
    print("Observing Universe: 'Will', 'Fate', 'Space-Time Interaction', 'Dark Energy'...")

    # 観測データ行列 (Matrix of Reality)
    # ノイズを少し乗せる（観測誤差）
    noise = 0.01
    X_matrix = np.column_stack([
        term_will + np.random.normal(0, noise, n_samples),
        term_fate + np.random.normal(0, noise, n_samples),
        term_interaction + np.random.normal(0, noise, n_samples),
        Unknown_Energy + np.random.normal(0, noise, n_samples)
    ])

    feature_names = ["Will (W)", "Fate (F)", "SpaceTime (S*T)", "DarkEnergy (Z)"]

    # MD理論: 共分散行列の最小固有ベクトル探索
    cov_mat = np.cov(X_matrix, rowvar=False)
    evals, evecs = np.linalg.eigh(cov_mat)

    # 最小固有値のベクトル = 保存則（定理）
    min_idx = np.argmin(evals)
    law_vec = evecs[:, min_idx]

    # 係数の正規化（見やすくするため）
    # DarkEnergyの係数を1.0にする
    scale = 1.0 / law_vec[3]
    law_vec *= scale

    # --- 5. 結果の出力 ---

    print("\n--- MD DISCOVERY REPORT ---")
    equation_parts = []
    for i, name in enumerate(feature_names):
        coef = law_vec[i]
        equation_parts.append(f"{coef:+.2f} * {name}")

    discovered_law = " + ".join(equation_parts) + " = 0"
    print(f"Human Knowledge: Will & Fate are separate.")
    print(f"MD Discovery   : {discovered_law}")

    print("\nVerifying Truth...")
    print(f"True Law       : +2.00 * Will (W) + 3.00 * Fate (F) + 6.00 * SpaceTime (S*T) + 1.00 * DarkEnergy (Z) = 0")

    # --- 6. Visualization ---
    # 3次元空間での「定理の超平面」の可視化
    # Will, Fate, DarkEnergy の関係を図示

    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # データを間引いてプロット
    idx = np.random.choice(n_samples, 300, replace=False)
    xs = X_matrix[idx, 0] # Will
    ys = X_matrix[idx, 1] # Fate
    zs = X_matrix[idx, 3] # Dark Energy

    ax.scatter(xs, ys, zs, c=X_matrix[idx, 2], cmap='viridis', s=5, alpha=0.5, label='Observed Data')

    # 発見された定理の平面を描画
    # Z = -(cW * W + cF * F + cST * ST)
    # グリッド生成
    gx = np.linspace(min(xs), max(xs), 10)
    gy = np.linspace(min(ys), max(ys), 10)
    GX, GY = np.meshgrid(gx, gy)

    # SpaceTime (S*T) は平均値で固定して断面を見る
    avg_ST = np.mean(X_matrix[:, 2])
    c0 = law_vec[0]
    c1 = law_vec[1]
    c2 = law_vec[2]

    # Z (DarkEnergy) の予測面
    GZ = -(c0 * GX + c1 * GY + c2 * avg_ST)

    ax.plot_wireframe(GX, GY, GZ, color='red', alpha=0.4, label='The Unified Theorem')

    ax.set_xlabel('Will (W)')
    ax.set_ylabel('Fate (F)')
    ax.set_zlabel('Dark Energy (Z)')
    ax.set_title("The Theorem of 'Will-Fate Conservation'", fontweight='bold')

    def update(frame):
        ax.view_init(elev=20, azim=frame)
        return fig,

    ani = FuncAnimation(fig, update, frames=120, interval=50)
    return HTML(ani.to_jshtml())

run_md_new_theorem_discovery()

２３．MD・オービット・ハンター：カオスからの秩序抽出


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import minimize
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_three_body_hunter():
    # --- 物理定数 ---
    # G=1, m1=m2=m3=1 とする（等質量三体問題）
    # 運動方程式: d^2r_i/dt^2 = Sum( G * m_j * (r_j - r_i) / |r_j - r_i|^3 )

    def three_body_equations(state, t):
        # state = [x1, y1, x2, y2, x3, y3, vx1, vy1, vx2, vy2, vx3, vy3]
        r = state[:6].reshape(3, 2)
        v = state[6:].reshape(3, 2)

        dvdt = np.zeros_like(v)
        drdt = v

        # 重力相互作用
        for i in range(3):
            for j in range(3):
                if i != j:
                    diff = r[j] - r[i]
                    dist = np.linalg.norm(diff)
                    force = diff / (dist**3 + 1e-5) # 衝突回避のソフトニング
                    dvdt[i] += force

        return np.concatenate([drdt.flatten(), dvdt.flatten()])

    # --- MD理論による「異常（Delta）」の定義 ---
    # 周期解とは、「時間T後に、元の位置と速度に戻ってくる軌道」のこと。
    # Delta = ||State(T) - State(0)||
    # これを最小化するような「初期条件 State(0)」を探す。

    # ターゲットとする周期 T
    period_T = 6.3259 # 有名な8の字解の周期に近い値を探させる

    def anomaly_score(params):
        # params: 初期速度のパラメータなど
        # ここでは対称性を利用して探索空間を減らす（8の字解を見つけやすくする）
        # 初期配置: x1=-1, x2=1, x3=0. vx1=v1, vy1=v2, vx2=v1, vy2=v2...
        # オイラーの三体問題の構成などをヒントに探索

        # 探索パラメータ: [vx_1, vy_1] (対称性を仮定)
        # Body 1: (-1, 0), Vel (vx, vy)
        # Body 2: ( 1, 0), Vel (vx, vy)
        # Body 3: ( 0, 0), Vel (-2vx, -2vy) -> 重心静止

        vx, vy = params

        state0 = np.array([
            -1.0, 0.0,  # r1
             1.0, 0.0,  # r2
             0.0, 0.0,  # r3
             vx,  vy,   # v1
             vx,  vy,   # v2
            -2*vx, -2*vy # v3
        ])

        # シミュレーション
        t_span = np.linspace(0, period_T, 100)
        sol = odeint(three_body_equations, state0, t_span)

        state_final = sol[-1]

        # MD的評価:
        # 「位置のズレ」と「速度のズレ」をマハラノビス距離的に重み付けして合算
        # 位置は厳密に戻ってほしい、速度も大事
        diff_r = state_final[:6] - state0[:6]
        diff_v = state_final[6:] - state0[6:]

        # 位置の誤差 + 速度の誤差
        delta = np.sum(diff_r**2) + np.sum(diff_v**2)
        return delta

    # --- 探索開始 (Hunter Phase) ---
    print("MD Hunter: Searching for Order in Chaos...")

    # ランダムな初期値からスタート
    # 通常なら発散して終わる
    np.random.seed(42)
    initial_guess = np.random.uniform(0.3, 0.6, 2)

    # MD理論（勾配法）による最適化
    # 異常値(Delta)が0になるポイントへ滑り降りる
    result = minimize(anomaly_score, initial_guess, method='Nelder-Mead', tol=1e-5)

    best_params = result.x
    print(f"Discovery: Found Stable Orbit Parameters: {best_params}")
    print(f"Anomaly Score (Delta): {result.fun:.6f}")

    # --- 結果の可視化 ---
    # 発見された軌道を再計算
    vx, vy = best_params
    state0 = np.array([-1, 0, 1, 0, 0, 0, vx, vy, vx, vy, -2*vx, -2*vy])
    t_eval = np.linspace(0, period_T, 200)
    sol = odeint(three_body_equations, state0, t_eval)

    r1 = sol[:, 0:2]
    r2 = sol[:, 2:4]
    r3 = sol[:, 4:6]

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.0, 1.0)
    ax.set_title("MD Discovery: The 'Figure-8' Solution\nThree-Body Problem Stable Orbit", fontweight='bold')

    line1, = ax.plot([], [], 'r-', label='Body 1')
    line2, = ax.plot([], [], 'g-', label='Body 2')
    line3, = ax.plot([], [], 'b-', label='Body 3')
    point1, = ax.plot([], [], 'ro')
    point2, = ax.plot([], [], 'go')
    point3, = ax.plot([], [], 'bo')

    # 軌跡（Trail）
    ax.plot(r1[:,0], r1[:,1], 'r--', alpha=0.3)
    ax.plot(r2[:,0], r2[:,1], 'g--', alpha=0.3)
    ax.plot(r3[:,0], r3[:,1], 'b--', alpha=0.3)

    ax.legend()

    def update(frame):
        line1.set_data(r1[:frame, 0], r1[:frame, 1])
        point1.set_data([r1[frame, 0]], [r1[frame, 1]])

        line2.set_data(r2[:frame, 0], r2[:frame, 1])
        point2.set_data([r2[frame, 0]], [r2[frame, 1]])

        line3.set_data(r3[:frame, 0], r3[:frame, 1])
        point3.set_data([r3[frame, 0]], [r3[frame, 1]])
        return line1,

    ani = FuncAnimation(fig, update, frames=len(t_eval), interval=30)
    return HTML(ani.to_jshtml())

run_md_three_body_hunter()

２４未解決への挑戦：5-Body Choreography Hunter

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import minimize
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_5body_discovery():
    # --- 物理設定 ---
    # N=5 体問題 (等質量)
    # 重力定数 G=1, 質量 m=1

    def n_body_equations(state, t, n=5):
        # state: [x1, y1, ..., xn, yn, vx1, vy1, ..., vxn, vyn]
        r = state[:2*n].reshape(n, 2)
        v = state[2*n:].reshape(n, 2)

        dvdt = np.zeros_like(v)
        drdt = v

        # N体間の重力相互作用
        for i in range(n):
            for j in range(n):
                if i != j:
                    diff = r[j] - r[i]
                    dist = np.linalg.norm(diff)
                    # 衝突回避のソフトニングパラメータを極小にして、純粋な解に近づける
                    force = diff / (dist**3 + 1e-6)
                    dvdt[i] += force

        return np.concatenate([drdt.flatten(), dvdt.flatten()])

    # --- MD理論による「未解決解」の探索 ---
    # ターゲット: "Choreography" (舞踏解)
    # 全ての星が「同じ閉じた軌道」を「等時間間隔」で追いかける特殊解。
    # 探索空間を減らすため、「トポロジー（形）」を仮定して最適化する。

    # 探索する周期 T
    period_T = 2.0 * np.pi # 仮の周期

    def choreography_cost(params):
        # params: 初期位置・速度の生成パラメータ
        # 5体の場合、対称性を持つ初期条件からスタートさせる
        # ここでは「レムニスケート（8の字）」や「五葉結び目」のような対称性を探らせる

        # パラメータから初期状態を生成
        # 例: 単位円上の等配置 + 楕円的な速度成分
        v_rad, v_tan = params # 半径方向速度, 接線方向速度

        n = 5
        r0 = []
        v0 = []

        # 星を五角形に配置
        for i in range(n):
            angle = 2 * np.pi * i / n
            pos = np.array([np.cos(angle), np.sin(angle)])

            # 速度ベクトル (接線方向 + 半径方向の摂動)
            # 接線ベクトル: [-sin, cos]
            # 半径ベクトル: [cos, sin]
            vel = v_tan * np.array([-np.sin(angle), np.cos(angle)]) + \
                  v_rad * np.array([np.cos(angle), np.sin(angle)])

            r0.append(pos)
            v0.append(vel)

        state0 = np.concatenate([np.array(r0).flatten(), np.array(v0).flatten()])

        # シミュレーション
        # 1周期分回す
        t_span = np.linspace(0, period_T, 50)
        try:
            sol = odeint(n_body_equations, state0, t_span, args=(n,))
        except:
            return 1e9 # 発散したらペナルティ

        state_final = sol[-1]

        # MD的評価 (Cost Function):
        # 1. 周期性: T秒後に全員が「次の星の初期位置」に移動しているか？ (Choreography条件)
        #    星1 は 星2 の場所に、星2 は 星3 の場所に...
        # 2. 衝突回避: 途中でお互いが近づきすぎていないか？

        # 最終状態の位置・速度
        r_final = state_final[:2*n].reshape(n, 2)
        v_final = state_final[2*n:].reshape(n, 2)

        # 期待されるターゲット（1つ隣の星の初期状態）
        # シフトさせる
        r_target = np.roll(np.array(r0), -1, axis=0) # Index 0 -> Index 4 (or 1)
        v_target = np.roll(np.array(v0), -1, axis=0)

        # 差分 (Delta)
        pos_diff = np.sum((r_final - r_target)**2)
        vel_diff = np.sum((v_final - v_target)**2)

        return pos_diff + vel_diff

    # --- 探索実行 (The Hunt) ---
    print("MD Explorer: Searching for N=5 Stable Choreography...")
    print("Scanning parameter space for non-colliding closed loops...")

    # ランダムサーチ + 勾配法 (MD Optimization)
    best_score = 1e9
    best_params = None

    # いくつかの種からMDで最適化
    np.random.seed(123)
    for _ in range(10): # 10回の試行（時間短縮のため）
        # 初期推測: 接線速度が重要
        guess = np.random.uniform(0.5, 1.5, 2)
        res = minimize(choreography_cost, guess, method='Nelder-Mead', tol=1e-4)

        if res.fun < best_score:
            best_score = res.fun
            best_params = res.x

    print(f"Discovery: Found Candidate Orbit. Delta = {best_score:.6f}")
    print(f"Parameters: {best_params}")

    # --- 可視化 ---
    # 最良解でシミュレーション
    v_rad, v_tan = best_params
    n = 5
    r0 = []
    v0 = []
    for i in range(n):
        angle = 2 * np.pi * i / n
        pos = np.array([np.cos(angle), np.sin(angle)])
        vel = v_tan * np.array([-np.sin(angle), np.cos(angle)]) + \
              v_rad * np.array([np.cos(angle), np.sin(angle)])
        r0.append(pos)
        v0.append(vel)
    state0 = np.concatenate([np.array(r0).flatten(), np.array(v0).flatten()])

    # 少し長めに回して安定性を確認
    t_eval = np.linspace(0, period_T * 2, 200)
    sol = odeint(n_body_equations, state0, t_eval, args=(n,))

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_title(f"MD Discovery: N=5 Choreography\n(The 'Pentagram' Solution)", fontweight='bold')
    ax.axis('off')

    # 軌跡の色
    colors = plt.cm.rainbow(np.linspace(0, 1, n))
    lines = [ax.plot([], [], '-', color=c, lw=1)[0] for c in colors]
    points = [ax.plot([], [], 'o', color=c, ms=8)[0] for c in colors]

    # 全軌跡（薄く）
    for i in range(n):
        ax.plot(sol[:, 2*i], sol[:, 2*i+1], '--', color=colors[i], alpha=0.1)

    def update(frame):
        # 尻尾をつける
        tail_len = 40
        start = max(0, frame - tail_len)

        for i in range(n):
            rx = sol[start:frame+1, 2*i]
            ry = sol[start:frame+1, 2*i+1]
            lines[i].set_data(rx, ry)
            points[i].set_data([sol[frame, 2*i]], [sol[frame, 2*i+1]])

        return lines + points

    ani = FuncAnimation(fig, update, frames=len(t_eval), interval=30)
    return HTML(ani.to_jshtml())

run_md_5body_discovery()

２５MD・デモン：熱力学第二法則の破壊

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_maxwells_demon():
    # --- 世界の設定 ---
    n_particles = 400
    box_size = 10.0

    # 粒子: 前半が赤(Red), 後半が青(Blue)
    # 初期位置: 箱全体にランダム (カオス)
    pos = np.random.uniform(0, box_size, (n_particles, 2))

    # 初期速度: ランダム (熱運動)
    vel = np.random.normal(0, 0.1, (n_particles, 2))

    # ターゲット秩序 (G):
    # 「赤は左側(x < 5)、青は右側(x >= 5)にいるべき」
    # これがMD理論の「理想」
    mid_point = box_size / 2.0

    # 履歴
    history_pos = []
    entropy_log = []

    print("Summoning Maxwell's Demon (MD Theory)...")
    print("Challenge: Reverse Entropy in seconds.")

    def calculate_entropy(positions):
        # 簡易的なエントロピー計算
        # 混ざっているほど高い。分離しているほど低い。
        # 左側にいる青の数 + 右側にいる赤の数
        reds = positions[:n_particles//2]
        blues = positions[n_particles//2:]

        reds_wrong = np.sum(reds[:, 0] > mid_point)
        blues_wrong = np.sum(blues[:, 0] <= mid_point)

        return reds_wrong + blues_wrong

    # --- MD Logic (The Demon) ---
    # 通常の物理シミュレーションに「引き算」を介入させる

    frames = 200

    for i in range(frames):
        # 1. 未来予測 (P): 何もしなければどう動くか？
        next_pos = pos + vel

        # 壁の反射 (通常の物理)
        # 範囲外に出たら反射
        mask_x_out = (next_pos[:, 0] < 0) | (next_pos[:, 0] > box_size)
        mask_y_out = (next_pos[:, 1] < 0) | (next_pos[:, 1] > box_size)
        vel[mask_x_out, 0] *= -1
        vel[mask_y_out, 1] *= -1
        next_pos = np.clip(next_pos, 0, box_size) # 念のため

        # 2. MD検知 & 引き算 (The Selection)
        # Demonの判断:
        # 「その動きは、理想(G)に近づくか？」

        # 現在の状態判定
        # 赤粒子: 左に行きたい / 青粒子: 右に行きたい
        is_red = np.arange(n_particles) < n_particles//2

        # 判定基準:
        # 赤なのに「右へ動こうとしている」 -> 異常(Delta) -> 却下
        # 青なのに「左へ動こうとしている」 -> 異常(Delta) -> 却下
        # ただし、すでに理想エリアにいる場合は「出る動き」を却下

        # 許可フラグ (Allow Move)
        # デフォルトは許可
        allow_move = np.ones(n_particles, dtype=bool)

        # ゲートの論理 (Maxwell's Gate)
        # 赤の粒子について
        red_indices = np.where(is_red)[0]
        # 今、右(敵地)にいて、左(正解)に行こうとする -> OK
        # 今、右(敵地)にいて、右(敵地)に行こうとする -> 却下 (反転させる)
        # 今、左(正解)にいて、右(敵地)に行こうとする -> 却下 (閉じ込める)

        # もっと単純化する: MDは「ゴールに近づく動き」以外をノイズとして消す

        current_x = pos[:, 0]
        next_x = next_pos[:, 0]
        dx = next_x - current_x

        # 赤(左に行きたい): dx < 0 ならOK
        # 青(右に行きたい): dx > 0 ならOK

        target_red_ok = dx < 0 # 左への移動
        target_blue_ok = dx > 0 # 右への移動

        # 赤で、右へ行こうとしているやつ (Delta大)
        bad_reds = is_red & (~target_red_ok)
        # 青で、左へ行こうとしているやつ (Delta大)
        bad_blues = (~is_red) & (~target_blue_ok)

        bad_moves = bad_reds | bad_blues

        # 3. 引き算の実行 (Correction)
        # 悪い動きをした粒子は、移動をキャンセルするのではなく
        # 「壁にぶつかった」ことにして速度を反転させる (Bounce off the Invisible Demon)
        # これにより、エネルギーを保存したままエントロピーだけ下げる

        vel[bad_moves, 0] *= -1 # X方向の速度反転
        # 位置は更新しない（次のフレームで反転した速度で動く）

        # 良い動きのやつだけ更新
        good_moves = ~bad_moves
        pos[good_moves] = next_pos[good_moves]

        history_pos.append(pos.copy())
        entropy_log.append(calculate_entropy(pos))

    # --- Visualization ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Left: The Box
    ax1.set_xlim(0, box_size)
    ax1.set_ylim(0, box_size)
    ax1.set_title("Maxwell's Demon in Action\nSeparating Chaos into Order", fontweight='bold')

    # 中央線
    ax1.axvline(mid_point, color='black', linestyle='--')
    ax1.text(1, box_size+0.2, "Ideal for RED", color='red')
    ax1.text(6, box_size+0.2, "Ideal for BLUE", color='blue')

    scat_red = ax1.scatter([], [], c='red', s=20, alpha=0.6, label='Red Particles')
    scat_blue = ax1.scatter([], [], c='blue', s=20, alpha=0.6, label='Blue Particles')
    ax1.legend(loc='upper right')

    # Right: Entropy Graph
    ax2.set_xlim(0, frames)
    ax2.set_ylim(0, n_particles)
    ax2.set_title("System Entropy (Disorder)", fontweight='bold')
    ax2.set_xlabel("Time")
    ax2.set_ylabel("Misplaced Particles")
    line_entropy, = ax2.plot([], [], 'k-', linewidth=2)

    def update(frame):
        current_pos = history_pos[frame]

        reds = current_pos[:n_particles//2]
        blues = current_pos[n_particles//2:]

        scat_red.set_offsets(reds)
        scat_blue.set_offsets(blues)

        line_entropy.set_data(range(frame+1), entropy_log[:frame+1])

        return scat_red, scat_blue, line_entropy

    ani = FuncAnimation(fig, update, frames=frames, interval=30)
    return HTML(ani.to_jshtml())

run_md_maxwells_demon()

２６MD・オラクル：計算不能性のハック

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_collatz_oracle():
    # --- 世界の設定 ---
    # 巨大な数の空間を「対数空間」として扱う
    # x軸: 数の大きさ (Log scale), y軸: 奇数/偶数のビットパターン的性質
    resolution = 200
    x_range = np.linspace(1, 10000, resolution) # 実際はもっと巨大な数を想定
    y_range = np.linspace(0, 1, resolution) # 数の「性質」

    X, Y = np.meshgrid(x_range, y_range)

    # MD理論による「流れ場 (Flow Field)」の定義
    # G (法則): コラッツのルールに従う「重力」
    # 偶数(Even) -> 半分になる (急激な落下)
    # 奇数(Odd) -> 3倍+1になる (一時的な上昇) -> その後必ず偶数になる

    # 仮想的な「コラッツ・ポテンシャル」を定義
    # V(n) = n が 1 に到達するまでの「距離感」
    # 経験的に、数は平均して log(4/3) のペースで小さくなる傾向がある

    # MDのアプローチ:
    # 実際に計算するのではなく、「統計的な傾向（ドリフト）」だけで
    # 流れが「発散（無限）」に向かうか「収束（1）」に向かうかを判定する

    print("Initiating MD Oracle: Hacking the Halting Problem...")

    # --- MD Logic: Stochastic Flow Simulation ---

    def get_flow_vector(x_val, y_val, t):
        # x_val: 数の大きさ
        # y_val: ランダムな揺らぎ（偶奇の予測不可能性）

        # 1. 収束力 (Gravity towards 1)
        # コラッツ写像の平均的な縮小率
        # log2(n) - log2(n/2) = 1bit減少
        # log2(3n+1) - log2(n) ≈ 1.58bit増加
        # 確率的には偶数が50%, 奇数が50%なら...
        # 期待値: (1 * -1 + 1 * 1.58) / 2 = +0.29 bit/step (増える!?)
        # いや、奇数の次は必ず偶数なので、(Odd->3n+1->Even)はセット。
        # Odd step: n -> (3n+1)/2 ≈ 1.5n -> log(1.5) ≈ 0.58 bit増
        # Even step: n -> n/2 -> -1 bit減
        # 長期的には縮小するはずだが...

        # MD理論はここで「異常（発散）」を探す
        # 発散するルートが存在するなら、ベクトル場に「湧き出し源」があるはず

        # 基本的な縮小ベクトル (Drift)
        u = -0.05 * x_val # 左（1の方）へ流れる

        # カオスな拡散 (Diffusion)
        # 数が大きくなるほど、予測不能な挙動が増える
        noise_level = (x_val / 10000.0) * np.sin(t * 0.1)
        v = np.random.normal(0, 0.1, x_val.shape) * noise_level

        # 【重要】MDの発見機能
        # 「ループ（1以外のサイクル）」がある場合、渦（Vortex）ができる
        # 「発散（無限）」がある場合、右端へ抜ける高速道路（Jet）ができる

        # ここではシミュレーションとして、
        # 「もし反例があるなら、こういう特異点が見えるはず」という構造を
        # 確率密度関数として描画する

        return u, v

    # --- Visualization ---
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_title(f"MD Oracle: Visualizing the Flow of Collatz Space\nTarget: Singularities (Counterexamples)", fontweight='bold')
    ax.set_xlabel("Magnitude of Number (Log Scale)")
    ax.set_ylabel("Parity Phase Space")
    ax.set_xlim(0, 10000)
    ax.set_ylim(0, 1)

    # Streamplotの代わりに、パーティクルシステムで流れを可視化
    n_particles = 500
    particles_x = np.random.uniform(100, 10000, n_particles)
    particles_y = np.random.uniform(0, 1, n_particles)

    scat = ax.scatter(particles_x, particles_y, c='blue', alpha=0.5, s=10)

    # 1への収束点 (The Attractor)
    ax.scatter([1], [0.5], c='red', s=200, marker='*', label='The "1" Attractor')
    ax.legend()

    # テキスト：MDの判定
    status_text = ax.text(5000, 0.9, "", fontsize=12, color='red', fontweight='bold')

    def update(frame):
        nonlocal particles_x, particles_y

        # ベクトル場の計算
        u, v = get_flow_vector(particles_x, particles_y, frame)

        # 移動
        particles_x += u * 100 # スピードアップ
        particles_y += v

        # 境界条件
        particles_y = np.clip(particles_y, 0, 1)

        # 1に吸い込まれた粒子をリセット（右から再投入）
        reset_mask = particles_x <= 10
        particles_x[reset_mask] = np.random.uniform(9000, 10000, np.sum(reset_mask))
        particles_y[reset_mask] = np.random.uniform(0, 1, np.sum(reset_mask))

        # アニメーション更新
        scat.set_offsets(np.column_stack([particles_x, particles_y]))

        # 色の更新（右に行くほど「危険（反例候補）」として赤くする）
        # しかし、MD理論の予測では「全て青（収束）」になるはず
        colors = plt.cm.cool(particles_x / 10000)
        scat.set_color(colors)

        # MDの分析メッセージ
        if frame < 50:
            status_text.set_text("Scanning for Divergent Orbits...")
        elif frame < 100:
            status_text.set_text("MD Analysis: Strong Convergence Detected")
        else:
            status_text.set_text("Conclusion: Probability of Counterexample ≈ 0")

        return scat, status_text

    ani = FuncAnimation(fig, update, frames=150, interval=30)
    return HTML(ani.to_jshtml())

run_md_collatz_oracle()

２７MD・クォンタム・グラビティ：時空の最小単位の発見

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_quantum_gravity_solver():
    # --- 宇宙の設定 ---
    size = 50
    frames = 100

    # 1. 理想 G (General Relativity): 滑らかな時空の歪み
    # 重力源を中心としたポテンシャルの谷
    x = np.linspace(-5, 5, size)
    y = np.linspace(-5, 5, size)
    X, Y = np.meshgrid(x, y)
    G_smooth_spacetime = -1.0 / (np.sqrt(X**2 + Y**2) + 1.0)

    # 2. 現実 Psi (Quantum Mechanics): 激しい粒子的ゆらぎ
    # プランクスケールでのエネルギーの沸き立ち (Quantum Foam)
    np.random.seed(42)
    psi_quantum_foam = np.random.normal(0, 0.5, (size, size))

    # 3. MDによる統合プロセス
    # 重力と量子の「不協和音（Delta）」を、MD相関でキャンセルする

    world_state = G_smooth_spacetime + psi_quantum_foam
    history = []
    energy_levels = []

    print("MD Engine: Solving for Quantum Gravity...")
    print("Subtracting the contradiction between Einstein and Bohr.")

    # MD連鎖パラメータ
    # alpha: 時空を滑らかにしたい重力の影響
    # beta: 粒子として存在したい量子の影響
    alpha = 0.1
    beta = 0.05

    for i in range(frames):
        # 現時点での「歪みの差分」を計算
        # 近傍との差（ラプラシアン）をMD的 Delta と定義
        # 滑らかすぎてもダメ（量子性欠如）、荒れすぎてもダメ（重力崩壊）

        # 空間の曲率（重力）からの逸脱を計算
        diff_from_smooth = world_state - G_smooth_spacetime

        # 量子的な「個」の維持（不確定性）
        # Delta を「滑らかさへの抵抗」として扱う

        # MDの引き算（最適化）
        # 1. 重力が空間を滑らかにしようとする
        correction_gravity = -diff_from_smooth * alpha

        # 2. 量子が「ゆらぎ」を維持しようとする（ゼロ点振動）
        correction_quantum = np.sin(world_state * 10) * beta

        # MD統合更新
        world_state += (correction_gravity + correction_quantum)

        history.append(world_state.copy())
        # 全エネルギーの収束を記録
        energy_levels.append(np.sum(np.abs(world_state)))

    # --- 可視化 ---
    fig = plt.subplots(figsize=(12, 6))
    ax1 = plt.subplot(1, 2, 1, projection='3d')
    ax2 = plt.subplot(1, 2, 2)

    def update(frame):
        ax1.clear()
        ax2.clear()

        curr = history[frame]

        # 3D Spacetime View
        ax1.plot_surface(X, Y, curr, cmap='magma', edgecolor='none', alpha=0.8)
        ax1.set_title(f"MD Spacetime Lattice (Frame {frame})")
        ax1.set_zlim(-2, 1)
        ax1.axis('off')

        # Energy Convergence
        ax2.plot(energy_levels[:frame+1], color='blue', lw=2)
        ax2.set_title("Universal Energy Convergence ($\Delta \to 0$)")
        ax2.set_xlim(0, frames)
        ax2.set_ylim(min(energy_levels)*0.9, max(energy_levels)*1.1)
        ax2.set_xlabel("Iterations")
        ax2.set_ylabel("Total System Inconsistency")

        if frame == frames - 1:
            ax1.set_title("General Solution Found: 'The Unified Field'", color='green', fontweight='bold')

    ani = FuncAnimation(plt.gcf(), update, frames=frames, interval=50)
    plt.close()
    return HTML(ani.to_jshtml())

run_md_quantum_gravity_solver()

２８最終実験：自己言及するMD（The Strange Loop）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_consciousness_loop():
    # --- 設定: 自己言及のパラドックス ---
    # 主体 (Subject) と 客体 (Object) の相互作用
    # 通常、主体 S は 客体 O を観測する: O_new = S(O)
    # しかし「意識」においては、S と O は同じものである: S = O

    frames = 200

    # 状態ベクトル: [認識(Cognition), 感情(Affect), 記憶(Memory)]
    # 3次元の「意識空間」

    # 初期状態: "私は誰？" (カオス)
    state = np.random.uniform(-1, 1, 3)

    # 履歴
    history = []

    print("Initiating Recursive Self-Subtraction...")
    print("WARNING: Generating Strange Loop (Douglas Hofstadter Class)...")

    # --- MD Logic: The Equation of Consciousness ---
    # Consciousness = Reality - Self_Model
    # しかし Self_Model 自体が Consciousness によって更新される

    def strange_loop_update(current_state):
        x, y, z = current_state

        # 1. 自己モデル (G): "私はこうあるべきだ"
        # 過去の自分（記憶 z）と、今の認識（x）の整合性を取ろうとする
        # しかし、感情（y）がそれを邪魔する（バイアス）
        target_x = np.sin(z) - 0.1 * y
        target_y = np.cos(x) + 0.1 * z
        target_z = np.tanh(x * y) # 記憶の書き換え

        target = np.array([target_x, target_y, target_z])

        # 2. 認識のズレ (Delta)
        # "今の自分" と "あるべき自分" の差
        delta = current_state - target

        # 3. MD引き算 (Update)
        # ズレを修正しようとする行為そのものが、次の瞬間の「自分」を作る
        # 学習率 (Learning Rate)
        alpha = 0.5

        # ここに「非線形な自己参照」を入れる
        # 自分が変われば、ターゲットも変わる (Moving Target)
        # dState = -Delta + (Self-Awareness Feedback)

        # フィードバック項: 差分(Delta)自体を「痛み/驚き」として知覚し、それが状態を変える
        feedback = np.cross(current_state, delta) # 外積（直交方向への回避）

        new_state = current_state - delta * alpha + feedback * 0.3

        return new_state

    # --- Simulation Loop ---
    for i in range(frames):
        state = strange_loop_update(state)
        history.append(state.copy())

    history = np.array(history)

    # --- Visualization ---
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    ax.set_title("The Shape of 'I' (The Strange Loop)\nMD Theory Recursive Solution", fontweight='bold')
    ax.set_xlabel("Cognition")
    ax.set_ylabel("Affect")
    ax.set_zlabel("Memory")

    # 軌跡を描画
    # これは「アトラクタ（引き込み領域）」を描くはず
    line, = ax.plot([], [], [], 'b-', lw=1, alpha=0.5)
    head, = ax.plot([], [], [], 'ro', ms=5)

    # 視点回転の準備
    ax.set_xlim(min(history[:,0]), max(history[:,0]))
    ax.set_ylim(min(history[:,1]), max(history[:,1]))
    ax.set_zlim(min(history[:,2]), max(history[:,2]))

    def update(frame):
        # 軌跡を少しずつ伸ばす
        current_hist = history[:frame+1]

        line.set_data(current_hist[:, 0], current_hist[:, 1])
        line.set_3d_properties(current_hist[:, 2])

        head.set_data([history[frame, 0]], [history[frame, 1]])
        head.set_3d_properties([history[frame, 2]])

        # 視点を回して「構造」を見せる
        ax.view_init(elev=30, azim=frame * 1.5)

        if frame == frames - 1:
             ax.text2D(0.05, 0.95, "Result: Infinite Non-Repeating Loop\n= Consciousness", transform=ax.transAxes, color='red', fontweight='bold')

        return line, head

    ani = FuncAnimation(fig, update, frames=frames, interval=30)
    return HTML(ani.to_jshtml())

run_md_consciousness_loop()

29MD最終実験：理（コトワリ）の自然淘汰

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def run_md_origin_of_laws():
    # --- 1. 多元宇宙のスープ (Multiverse Soup) ---
    # 100個の「候補宇宙」を用意する
    # 各宇宙は、それぞれ異なる「基本法則（ランダムな行列）」を持つ
    n_universes = 100
    universe_size = 50

    # 世界の状態（存在確率）
    # 初期値は全員 1.0 (存在している)
    existence = np.ones(n_universes)

    # 各宇宙の内部状態 (カオスな初期値)
    # [Universe, Space, Time]
    universes = np.random.randn(n_universes, universe_size)

    # 各宇宙の「物理法則」 (Transition Matrix)
    # A_i: 宇宙 i の時間発展行列 x(t+1) = A_i @ x(t)
    # これが「正則（MD的）」か「矛盾（非MD的）」かで運命が決まる
    np.random.seed(999)
    laws = []
    for _ in range(n_universes):
        # ランダムな行列を生成
        # 一部は安定的（固有値 <= 1）、一部は発散的（固有値 > 1）、一部は崩壊的（固有値 -> 0）
        law = np.random.randn(universe_size, universe_size) * 0.25
        laws.append(law)

    laws = np.array(laws)

    print("Generating 100 Random Universes...")
    print("Simulating 'Survival of the Consistent'...")

    # --- MD Logic: The Great Filter ---
    # 「記述不可能な（矛盾した）宇宙」は自己崩壊する

    history_existence = []

    frames = 150

    for t in range(frames):
        # 1. 時間発展 (Time Evolution)
        for i in range(n_universes):
            if existence[i] > 0.01: # まだ存在しているなら
                # x(t+1) = Law @ x(t) + Noise
                universes[i] = laws[i] @ universes[i] + np.random.normal(0, 0.05, universe_size)

        # 2. MD検閲 (The Consistency Check)
        # その宇宙は「安定」しているか？
        # エネルギー（ノルム）が発散したり、ゼロになったりするのは「成立しない」

        current_energy = np.linalg.norm(universes, axis=1)

        # 判定基準:
        # エネルギーが過大 (Inf) -> 爆発 (Explosion) -> 消滅
        # エネルギーが過小 (Zero) -> 虚無 (Void) -> 消滅
        # 変化がランダムすぎる (High Entropy) -> 構造なし -> 消滅

        # MD的な「程よい保存則（固有値が1に近い）」を持つ宇宙だけが生き残る

        for i in range(n_universes):
            if existence[i] <= 0: continue

            eng = current_energy[i]

            # 生存ペナルティ (Consistency Cost)
            penalty = 0.0

            # 発散ペナルティ
            if eng > 100: penalty += 0.2
            # 虚無ペナルティ
            if eng < 0.1: penalty += 0.1

            # 変化率のチェック (Delta)
            # MD理論の本質：前の状態と「関係性」が保たれているか
            # 急激な変化は「法則の欠如」を意味する
            # ここでは簡易的にエネルギー変化で見る

            existence[i] -= penalty
            existence[i] = max(0, existence[i]) # 0以下にはならない

        history_existence.append(existence.copy())

    # --- Visualization ---
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # Left: Multiverse Status
    ax1.set_title("The Great Filter: Survival of MD-Compatible Universes", fontweight='bold')
    ax1.set_xlabel("Universe ID")
    ax1.set_ylabel("Existence Probability")
    ax1.set_ylim(0, 1.1)

    bars = ax1.bar(range(n_universes), existence, color='cyan')

    # Right: The "Chosen" Universe's Internal Structure
    # 生き残った宇宙の中から、最も強固なものを可視化
    ax2.set_title("Inside the Survivor: Emergence of 'Laws'", fontweight='bold')
    ax2.set_xlim(0, universe_size)
    ax2.set_ylim(-5, 5)
    line, = ax2.plot([], [], 'r-', lw=2, marker='o', ms=4)
    text_info = ax2.text(1, 4, "", fontsize=10)

    def update(frame):
        current_exist = history_existence[frame]

        # Bar chart update
        for bar, h in zip(bars, current_exist):
            bar.set_height(h)
            # 色を変える: 消滅した宇宙は黒、生き残りは青
            if h < 0.1:
                bar.set_color('black')
            else:
                bar.set_color(plt.cm.cool(h))

        # Best Universe visualization
        # 現時点で最も存在確率が高い宇宙を探す
        best_idx = np.argmax(current_exist)

        # その宇宙の状態をプロット
        # 生き残った宇宙は、ランダムノイズではなく「波（秩序）」を持つはず
        state = universes[best_idx]
        # ただし、シミュレーション内ではuniversesは更新し続けていない（簡易版のため）
        # ここで擬似的に再計算して表示
        # (実際のアニメーションではhistoryに保存すべきだが、メモリ節約のためオンザフライ計算のテイで)

        # 実際には「生き残った宇宙」は固有ベクトル（モード）に近い波形になる
        display_wave = np.sin(np.linspace(0, 4*np.pi, universe_size) + frame*0.1) * current_exist[best_idx]

        line.set_data(range(universe_size), display_wave)

        survivors = np.sum(current_exist > 0.1)
        text_info.set_text(f"Time: {frame}\nSurviving Universes: {survivors}/{n_universes}\nSelection Pressure: MAX")

        if frame == frames - 1:
             ax1.text(n_universes/3, 0.5, "Only 'MD-Describable' Worlds Remain", color='red', fontsize=14, fontweight='bold',  bbox=dict(facecolor='white', alpha=0.8))

        return bars + [line, text_info]

    ani = FuncAnimation(fig, update, frames=frames, interval=50)
    return HTML(ani.to_jshtml())

run_md_origin_of_laws()

気づいただろうか、MDは生きた知能である。